# KSC: LLM-Driven **Composite** Test Generation (GPT‑4o, Python)
**Repo:** `temporalio/money-transfer-project-template-python`  
**LLM:** OpenAI GPT‑4o  
**Notebook 목적:** *분기 의무 + Def‑Use 체인 + 예외 경로*를 통합한 테스트를 자동 생성·실행·증분합니다.

In [1]:
#@title 3-0) 런타임 & 의존성 준비
import os, sys, subprocess, pathlib, getpass, shutil

# 1) 기본 상수/경로
REPO_URL = "https://github.com/temporalio/money-transfer-project-template-python.git"
PROJECT_NAME = REPO_URL.rstrip("/").split("/")[-1].replace(".git", "")
ROOT = pathlib.Path(".").resolve()
PROJ = ROOT / PROJECT_NAME

def sh(cmd: str, check: bool = True, cwd: pathlib.Path | None = None) -> None:
    print("> ", cmd)
    rc = subprocess.call(cmd, shell=True, cwd=str(cwd) if cwd else None)
    if check and rc != 0:
        raise RuntimeError(f"Command failed (rc={rc}): {cmd}")

print(f"Python {sys.version}")
print("ROOT:", ROOT)

# 2) 레포 클론/업데이트
if PROJ.exists():
    print(f"Repo exists at {PROJ}. Pulling latest…")
    sh(f"git -C {PROJ} fetch --all --prune")
    sh(f"git -C {PROJ} reset --hard origin/main")
else:
    sh(f"git clone --depth=1 {REPO_URL}")
print("✅ 레포 클론/업데이트 완료")

# 3) pip 의존성 설치
def pip_install(pkgs):
    cmd = [sys.executable, "-m", "pip", "install", "-U"] + pkgs
    print("> ", " ".join(cmd))
    subprocess.run(cmd, check=True)

pkgs_core = [
    "pytest==7.4.4", "pytest-asyncio==0.23.7", "pytest-mock==3.11.1", "pytest-cov==4.1.0",
    "coverage[toml]==7.6.1",
    "temporalio==1.7.0",
    "lxml==5.2.2",
    "asttokens==2.4.1", "libcst==1.4.0", "networkx==3.3",
    "rich==13.7.1", "pyyaml==6.0.2",
]
if sys.version_info < (3,11):
    pkgs_core.append("tomli")

pkgs_llm = [
    "openai==1.43.0",
    "httpx==0.27.2",
    "backoff==2.2.1",
    "tiktoken==0.7.0",
    "aiolimiter==1.1.0",
    "anyio==4.4.0",
    "nest_asyncio==1.6.0",
    "python-dotenv==1.0.1",
    "tqdm==4.66.5",
]

pip_install(["pip"])
pip_install(pkgs_core)
pip_install(pkgs_llm)
print("✅ 의존성 설치 완료")

# 4) .env 생성
try:
    from dotenv import load_dotenv
except:
    pip_install(["python-dotenv>=1.0.1"])
    from dotenv import load_dotenv

openai_api_key = getpass.getpass("Enter OPENAI_API_KEY (필수): ").strip()
env_path = ROOT / ".env"
env_path.write_text(f"OPENAI_API_KEY={openai_api_key}\n", encoding="utf-8")
load_dotenv(env_path)
os.environ["OPENAI_API_KEY"] = openai_api_key
print("✅ .env 저장 및 환경 주입 완료")

# 5) PYTHONPATH 설정
os.environ["PROJECT_PATH"] = str(PROJ)
py_paths = [str(PROJ)]
if (PROJ / "src").exists():
    py_paths.insert(0, str(PROJ / "src"))
prev_pp = os.environ.get("PYTHONPATH","")
os.environ["PYTHONPATH"] = ":".join(py_paths + ([prev_pp] if prev_pp else []))

print("✅ PYTHONPATH =", os.environ["PYTHONPATH"])

# 6) A안: coverage source 전체(.) 고정하여 .coveragerc 생성
coveragerc_text = """
[run]
branch = True
source =
    .
omit =
    tests/*
    generated_tests/*
    run_artifacts/*
    htmlcov/*
    .venv/*
    */site-packages/*

[report]
show_missing = True
skip_covered = True
"""
(PROJ / ".coveragerc").write_text(coveragerc_text.strip() + "\n", encoding="utf-8")
os.environ["COVERAGE_RCFILE"] = str(PROJ / ".coveragerc")

print("✅ .coveragerc 생성 완료")
print("  - source = . (프로젝트 전체)")
print("  - omit   = tests/, generated_tests/, run_artifacts/, htmlcov/, .venv, site-packages")

# 7) 결과 디렉터리 준비
for d in ["run_artifacts", "htmlcov", "reports"]:
    (PROJ / d).mkdir(parents=True, exist_ok=True)

print("✅ 결과 디렉토리 준비 완료")


Python 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
ROOT: /content
>  git clone --depth=1 https://github.com/temporalio/money-transfer-project-template-python.git
✅ 레포 클론/업데이트 완료
>  /usr/bin/python3 -m pip install -U pip
>  /usr/bin/python3 -m pip install -U pytest==7.4.4 pytest-asyncio==0.23.7 pytest-mock==3.11.1 pytest-cov==4.1.0 coverage[toml]==7.6.1 temporalio==1.7.0 lxml==5.2.2 asttokens==2.4.1 libcst==1.4.0 networkx==3.3 rich==13.7.1 pyyaml==6.0.2
>  /usr/bin/python3 -m pip install -U openai==1.43.0 httpx==0.27.2 backoff==2.2.1 tiktoken==0.7.0 aiolimiter==1.1.0 anyio==4.4.0 nest_asyncio==1.6.0 python-dotenv==1.0.1 tqdm==4.66.5
✅ 의존성 설치 완료
Enter OPENAI_API_KEY (필수): ··········
✅ .env 저장 및 환경 주입 완료
✅ PYTHONPATH = /content/money-transfer-project-template-python:/env/python
✅ .coveragerc 생성 완료
  - source = . (프로젝트 전체)
  - omit   = tests/, generated_tests/, run_artifacts/, htmlcov/, .venv, site-packages
✅ 결과 디렉토리 준비 완료


In [2]:
#@title 3-1) 기준선 측정 (실행 가능한 라인/브랜치 기반)

import os, sys, json, subprocess, shutil, re
from pathlib import Path
from lxml import etree

# ====== 0) 경로/환경 ======
assert "PROJ" in globals(), "3-0 단계가 먼저 실행되어야 합니다."
PROJ = Path(PROJ).resolve()
ART_DIR = PROJ / "run_artifacts" / "run1"
HTML_DIR = PROJ / "htmlcov"
ART_DIR.mkdir(parents=True, exist_ok=True)
HTML_DIR.mkdir(parents=True, exist_ok=True)

RCFILE = PROJ / ".coveragerc"
rc_opt = f" --rcfile {RCFILE}" if RCFILE.exists() else ""

def sh(cmd, check=False):
    print("> ", cmd)
    rc = subprocess.call(cmd, shell=True, cwd=str(PROJ))
    if check and rc != 0:
        raise RuntimeError(f"Command failed (rc={rc}): {cmd}")
    return rc

def norm(fp):
    p = Path(fp)
    if not p.is_absolute():
        p = (PROJ / p).resolve()
    return str(p)

def in_proj(abs_path):
    return str(PROJ) in str(Path(abs_path).resolve())

# ====== 1) baseline 테스트 실행 ======
print("🧪 BASELINE 테스트 실행 중…")
sh("coverage erase" + rc_opt)
sh(f"{sys.executable} -m coverage run{rc_opt} -m pytest -q")
sh(f"coverage json -o coverage_base.json{rc_opt}")
sh(f"coverage xml  -o coverage_base.xml{rc_opt}")
sh("coverage html" + rc_opt)

# 복사
json_path = PROJ / "coverage_base.json"
xml_path  = PROJ / "coverage_base.xml"
shutil.copy2(json_path, ART_DIR / "coverage_base.json")
shutil.copy2(xml_path, ART_DIR / "coverage_base.xml")

# ====== 2) baseline JSON 파싱 ======
cov = json.loads(json_path.read_text(encoding="utf-8"))
files = cov.get("files", {}) or {}

# 실행 가능 라인 계산 = executed + missing
def sum_len(key):
    return sum(len((files.get(f, {}) or {}).get(key, []) or []) for f in files)

base_exec = sum_len("executed_lines")
base_miss = sum_len("missing_lines")
total_executable_lines = base_exec + base_miss

# ====== 3) branch coverage (coverage XML → arc 기반) ======
full = half = zero = 0
observed_outcomes = {}

xml_root = etree.parse(str(xml_path)).getroot()
for cls in xml_root.findall(".//class"):
    filename = cls.get("filename")
    abs_f = norm(filename)
    if not in_proj(abs_f):
        continue

    for line in cls.findall("./lines/line"):
        if line.get("branch") != "true":
            continue

        num = int(line.get("number"))
        cond = line.get("condition-coverage")
        m = re.search(r"\((\d+)\s*/\s*(\d+)\)", cond) if cond else None
        if not m:
            continue

        covered, total = int(m.group(1)), int(m.group(2))
        observed_outcomes.setdefault(abs_f, {})[num] = {
            "covered": covered,
            "total": total,
            "ratio": round(covered / total, 3)
        }

        if covered == 0:
            zero += 1
        elif covered == total:
            full += 1
        else:
            half += 1

# 실행 가능한 branch 수
branch_points = full + half + zero

# ====== 4) uncovered_map 생성 ======
uncovered_map = {}
for f, finfo in files.items():
    abs_f = norm(f)
    if not in_proj(abs_f):
        continue
    if any(seg in abs_f for seg in ["tests/", "generated_tests/", "run_artifacts/", "htmlcov/"]):
        continue

    miss = finfo.get("missing_lines", []) or []
    if miss:
        uncovered_map[abs_f] = sorted(set(miss))

(ART_DIR / "uncovered_map_base.json").write_text(
    json.dumps(uncovered_map, indent=2, ensure_ascii=False),
    encoding="utf-8"
)
(ART_DIR / "observed_outcomes_base.json").write_text(
    json.dumps(observed_outcomes, indent=2, ensure_ascii=False),
    encoding="utf-8"
)

# ====== 5) baseline_info 저장 (실행 가능한 라인/브랜치 기준) ======
baseline_info = {
    "total_executable_lines": total_executable_lines,
    "total_executable_branches": branch_points,
    "baseline_executed_lines": base_exec,
    "baseline_missing_lines": base_miss,
    "baseline_branch_full": full,
    "baseline_branch_half": half,
    "baseline_branch_zero": zero,
}

(ART_DIR / "baseline_info.json").write_text(
    json.dumps(baseline_info, indent=2, ensure_ascii=False),
    encoding="utf-8"
)

# ====== 6) 출력 ======
print("✅ 기준선(3-1) 생성 완료")
print(" - uncovered_map_base.json 생성됨")
print(" - observed_outcomes_base.json 생성됨")
print(" - baseline_info.json 생성됨")

print("\n📘 [Baseline Summary]")
print(f" • 전체 실행 가능한 라인수      : {total_executable_lines}")
print(f" • 전체 실행 가능한 분기 수     : {branch_points}")

print("\n📊 [Baseline Coverage]")
print(f" • executed={base_exec}, missing={base_miss}")
pct = round((base_exec / (base_exec + base_miss)) * 100, 2) if (base_exec + base_miss) else 0
print(f"   → 라인 커버리지 비율 : {pct}%")

print("\n🔀 [Baseline Branch Coverage]")
print(f" • full-hit  : {full}")
print(f" • half-hit  : {half}")
print(f" • zero-hit  : {zero}")
pct_b = round((full / branch_points) * 100, 2) if branch_points else 0
print(f"   → 브랜치 커버리지 비율 : {pct_b}%")


🧪 BASELINE 테스트 실행 중…
>  coverage erase --rcfile /content/money-transfer-project-template-python/.coveragerc
>  /usr/bin/python3 -m coverage run --rcfile /content/money-transfer-project-template-python/.coveragerc -m pytest -q
>  coverage json -o coverage_base.json --rcfile /content/money-transfer-project-template-python/.coveragerc
>  coverage xml  -o coverage_base.xml --rcfile /content/money-transfer-project-template-python/.coveragerc
>  coverage html --rcfile /content/money-transfer-project-template-python/.coveragerc
✅ 기준선(3-1) 생성 완료
 - uncovered_map_base.json 생성됨
 - observed_outcomes_base.json 생성됨
 - baseline_info.json 생성됨

📘 [Baseline Summary]
 • 전체 실행 가능한 라인수      : 156
 • 전체 실행 가능한 분기 수     : 20

📊 [Baseline Coverage]
 • executed=102, missing=54
   → 라인 커버리지 비율 : 65.38%

🔀 [Baseline Branch Coverage]
 • full-hit  : 16
 • half-hit  : 0
 • zero-hit  : 4
   → 브랜치 커버리지 비율 : 80.0%


In [3]:
#@title 3-2a) 기존 테스트 프루닝 – 테스트 함수 단위 (기준선 대비 고유 Δcoverage + 탐욕적 선택; 수집필터/폴백 강화)
import subprocess, json, os, shutil, re, sys
from pathlib import Path

# ---------- 0. 경로/전제 ----------
assert 'PROJ' in globals(), "3-0 단계를 먼저 실행해야 합니다."
PROJ = Path(PROJ).resolve()
ART_DIR = PROJ / "run_artifacts" / "run1"
SRC_TESTS = PROJ / "tests"
DST_PRUNED = PROJ / "Pruned_Base_Tests"
DST_PRUNED.mkdir(exist_ok=True)

RCFILE = PROJ / ".coveragerc"
assert RCFILE.exists(), ".coveragerc가 없습니다. 3-0 단계를 먼저 실행하세요."
rc_opt = f"--rcfile={RCFILE}"

BASE_JSON = ART_DIR / "coverage_base.json"
assert BASE_JSON.exists(), "기준선 coverage_base.json이 없습니다. 3-1 단계를 먼저 실행하세요."

# ---------- 1. 기준선 라인 집합 ----------
def load_executed_lines_from_json(json_path: Path) -> set[str]:
    if not json_path.exists():
        return set()
    data = json.loads(json_path.read_text(encoding="utf-8"))
    lines = set()
    for f, finfo in (data.get("files", {}) or {}).items():
        for ln in (finfo.get("executed_lines", []) or []):
            lines.add(f"{f}:{ln}")
    return lines

baseline_lines = load_executed_lines_from_json(BASE_JSON)
print(f"📊 기준선 라인 수: {len(baseline_lines)}")

# ---------- 2. 테스트 목록 수집 (nodeid) ----------
print("📋 pytest 테스트 수집 중 (--collect-only)...")
collect_out = subprocess.check_output(
    [sys.executable, "-m", "pytest", "--collect-only", "-q"],
    cwd=PROJ,
    stderr=subprocess.STDOUT,
).decode("utf-8", errors="ignore")

# nodeid로 간주: "path::..." 패턴만 허용 (안내문/총계 라인 필터)
nodeid_pat = re.compile(r".+::.+")
test_items = [ln.strip() for ln in collect_out.splitlines()
              if nodeid_pat.match(ln.strip())]
print(f"총 수집된 테스트 수: {len(test_items)}")

if not test_items:
    print("ℹ️ 수집된 테스트가 없습니다. 프루닝을 건너뜁니다.")
    (ART_DIR / "pruning_summary.json").write_text(
        json.dumps({
            "total_tests": 0, "retained": 0, "removed": 0,
            "retained_tests": [], "removed_tests": [],
        }, indent=2, ensure_ascii=False),
        encoding="utf-8"
    )
else:
    # ---------- 3. 개별 테스트 실행 → 실행 라인 집합 ----------
    def get_executed_lines_tmp() -> set[str]:
        tmp_json = PROJ / "coverage_tmp.json"
        return load_executed_lines_from_json(tmp_json)

    def run_test_item(item: str) -> tuple[bool, set[str]]:
        subprocess.run(f"coverage erase {rc_opt}", shell=True, cwd=PROJ)
        rc = subprocess.call(
            f"{sys.executable} -m coverage run {rc_opt} -m pytest -q {item}",
            shell=True, cwd=PROJ,
        )
        subprocess.run(f"coverage json -o coverage_tmp.json {rc_opt}", shell=True, cwd=PROJ)
        return (rc == 0), get_executed_lines_tmp()

    per_test_exec: list[tuple[str, bool, set[str]]] = []
    for i, item in enumerate(test_items, 1):
        print(f"[{i}/{len(test_items)}] ▶ 실행 중: {item}")
        ok, exec_set = run_test_item(item)
        per_test_exec.append((item, ok, exec_set))
        print(f"   → {'✅PASS' if ok else '❌FAIL'}, executed_lines={len(exec_set)}")

    # ---------- 4. 탐욕적 선택 (set-cover 유사) ----------
    covered_so_far = set(baseline_lines)
    candidates = [(it, s) for (it, ok, s) in per_test_exec if ok]

    THRESHOLD_MIN_GAIN = 1  # 고유 기여 라인 ≥ 1
    selected: list[str] = []

    while True:
        best = None
        best_gain_val = 0
        for it, s in candidates:
            gain_val = len(s - covered_so_far)
            if gain_val > best_gain_val:
                best_gain_val = gain_val
                best = (it, s)
        if not best or best_gain_val < THRESHOLD_MIN_GAIN:
            break
        it, s = best
        selected.append(it)
        covered_so_far |= s
        candidates = [(iit, ss) for (iit, ss) in candidates if iit != it]

    # ---- 폴백: 모두 0이면 PASS 중에서 최대 executed_lines 1개 유지
    if not selected:
        passables = [(it, s) for (it, ok, s) in per_test_exec if ok]
        if passables:
            it, s = max(passables, key=lambda x: len(x[1]))
            selected = [it]
            covered_so_far |= s
            print(f"ℹ️ 폴백 적용: '{it}' 1개 유지 (실행 라인 {len(s)}).")

    # ---------- 5. 결과 요약 ----------
    selected_set = set(selected)
    retained = [{"name": it, "success": True} for it in selected]
    removed = [{"name": it, "success": bool(ok)}
               for (it, ok, _s) in per_test_exec if it not in selected_set]

    summary = {
        "total_tests": len(test_items),
        "retained": len(retained),
        "removed": len(removed),
        "retained_tests": retained,
        "removed_tests": removed,
        "baseline_lines": len(baseline_lines),
        "final_covered_lines": len(covered_so_far),
        "incremental_gain_lines": len(covered_so_far - baseline_lines),
        "threshold_min_gain": THRESHOLD_MIN_GAIN,
    }
    (ART_DIR / "pruning_summary.json").write_text(
        json.dumps(summary, indent=2, ensure_ascii=False),
        encoding="utf-8"
    )

    # ---------- 6. 유지된 테스트만 복사 ----------
    retained_files = set()
    for nodeid in selected:
        file_part = nodeid.split("::", 1)[0]
        path_obj = (PROJ / file_part).resolve()
        if path_obj.exists():
            retained_files.add(path_obj)

    if SRC_TESTS.exists():
        copied = 0
        for test_file in sorted(SRC_TESTS.rglob("test_*.py")):
            if test_file.resolve() in retained_files:
                rel = test_file.relative_to(SRC_TESTS)
                dst = DST_PRUNED / rel
                dst.parent.mkdir(parents=True, exist_ok=True)
                shutil.copy2(test_file, dst)
                copied += 1
        print(f"📦 유지 파일 복사 완료: {copied}개 파일 → {DST_PRUNED}")
    else:
        print("ℹ️ tests/ 폴더가 없어 파일 복사는 건너뜁니다.")

    (ART_DIR / "retained_nodeids.txt").write_text("\n".join(selected), encoding="utf-8")

    print("✅ 테스트 함수 단위 프루닝 완료")
    print(" - pruning_summary.json  :", ART_DIR / "pruning_summary.json")
    print(" - retained_nodeids.txt  :", ART_DIR / "retained_nodeids.txt")
    print(" - 출력 디렉터리(선택 파일):", DST_PRUNED)


📊 기준선 라인 수: 102
📋 pytest 테스트 수집 중 (--collect-only)...
총 수집된 테스트 수: 3
[1/3] ▶ 실행 중: tests/test_run_worker.py::test_money_transfer
   → ✅PASS, executed_lines=91
[2/3] ▶ 실행 중: tests/test_run_worker.py::test_money_transfer_withdraw_insufficient_funds
   → ✅PASS, executed_lines=83
[3/3] ▶ 실행 중: tests/test_run_worker.py::test_money_transfer_withdraw_invalid_account
   → ✅PASS, executed_lines=79
ℹ️ 폴백 적용: 'tests/test_run_worker.py::test_money_transfer' 1개 유지 (실행 라인 91).
📦 유지 파일 복사 완료: 1개 파일 → /content/money-transfer-project-template-python/Pruned_Base_Tests
✅ 테스트 함수 단위 프루닝 완료
 - pruning_summary.json  : /content/money-transfer-project-template-python/run_artifacts/run1/pruning_summary.json
 - retained_nodeids.txt  : /content/money-transfer-project-template-python/run_artifacts/run1/retained_nodeids.txt
 - 출력 디렉터리(선택 파일): /content/money-transfer-project-template-python/Pruned_Base_Tests


In [4]:
#@title 3-2b) 복합 목표 생성
import ast
import json
from pathlib import Path
from typing import Dict, List, Tuple, Set, Optional

# ---------- 경로 ----------
PROJ_PATH = Path(PROJ).resolve()
ART_DIR    = PROJ_PATH / "run_artifacts" / "run1"
UNCOVERED_JSON = ART_DIR / "uncovered_map_base.json"
OBSERVED_JSON  = ART_DIR / "observed_outcomes_base.json"

assert UNCOVERED_JSON.exists(),  "uncovered_map_base.json이 없습니다. 3-1을 먼저 실행하세요."
assert OBSERVED_JSON.exists(),   "observed_outcomes_base.json이 없습니다. 3-1을 먼저 실행하세요."

# ===================== 유틸 =====================
def norm_abs(p: str) -> str:
    q = Path(p)
    if not q.is_absolute():
        q = (PROJ_PATH / q).resolve()
    return str(q.resolve())

def rel_from_proj(abs_path: str) -> str:
    try:
        return str(Path(abs_path).resolve().relative_to(PROJ_PATH))
    except Exception:
        return abs_path

def is_source(abs_path: str) -> bool:
    try:
        rel = Path(abs_path).resolve().relative_to(PROJ_PATH)
    except Exception:
        return False
    s = str(rel).replace("\\", "/")
    if not s.endswith(".py"):
        return False
    if s.startswith(("generated_tests/", "tests/", ".venv/", "venv/", "run_artifacts/", "htmlcov/")):
        return False
    return True

def try_unparse(node: ast.AST) -> Optional[str]:
    try:
        if hasattr(ast, "unparse"):
            return ast.unparse(node)
    except Exception:
        return None


# ===================== baseline 입력 로드 =====================
raw_uncovered = json.loads(UNCOVERED_JSON.read_text(encoding="utf-8"))
uncovered_map: Dict[str, List[int]] = {
    norm_abs(k): sorted(set(v))
    for k, v in raw_uncovered.items()
    if is_source(norm_abs(k))
}

raw_observed = json.loads(OBSERVED_JSON.read_text(encoding="utf-8"))
observed_outcomes: Dict[str, Dict[int, Dict[str, float]]] = {}

for k, mp in raw_observed.items():
    abs_k = norm_abs(k)
    if not is_source(abs_k):
        continue
    fixed = {}
    for ln_str, meta in mp.items():
        try:
            fixed[int(ln_str)] = meta
        except:
            continue
    observed_outcomes[abs_k] = fixed

# half-hit
half_hit_map: Dict[str, Set[int]] = {}
for fp, mapping in observed_outcomes.items():
    half = {ln for ln, meta in mapping.items()
            if 0 < int(meta.get("covered", 0)) < int(meta.get("total", 0))}
    if half:
        half_hit_map[fp] = half


# ===================== AST 스캔 =====================
MOD_REQS = {"requests"}
MOD_HTTPX = {"httpx"}
MOD_TEMPORAL = {"temporalio"}
MOD_SUBPROCESS = {"subprocess"}
TEMPORAL_PREFIXES = ("temporalio.client.", "temporalio.worker.", "temporalio.workflow.")

class FunctionInfo:
    def __init__(self, name, lineno):
        self.name = name or "<module>"
        self.lineno = lineno
        self.branches: List[int] = []
        self.defs: Dict[str, List[int]] = {}
        self.uses: Dict[str, List[int]] = {}
        self.exceptions: List[Tuple[str, int, dict]] = []
        self.side_effect_calls: List[Tuple[str, int]] = []

    def add_def(self, v, ln): self.defs.setdefault(v, []).append(ln)
    def add_use(self, v, ln): self.uses.setdefault(v, []).append(ln)


class ASTVisitor(ast.NodeVisitor):
    def __init__(self, file_rel):
        self.file_rel = file_rel
        self.stack: List[FunctionInfo] = []
        self.funcs: List[FunctionInfo] = []
        self.alias_to_module = {}
        self.symbol_to_module = {}

    def current(self):
        if not self.stack:
            if not self.funcs or self.funcs[0].name != "<module>":
                fi = FunctionInfo("<module>", 1)
                self.funcs.insert(0, fi)
            return self.funcs[0]
        return self.stack[-1]

    # imports
    def visit_Import(self, node):
        for alias in node.names:
            mod = alias.name
            asname = alias.asname or mod.split(".")[0]
            self.alias_to_module[asname] = mod
        self.generic_visit(node)

    def visit_ImportFrom(self, node):
        mod = node.module or ""
        for alias in node.names:
            self.symbol_to_module[alias.asname or alias.name] = mod
        self.generic_visit(node)

    # function defs
    def visit_FunctionDef(self, node):
        fi = FunctionInfo(node.name, node.lineno)
        self.stack.append(fi); self.generic_visit(node); self.stack.pop()
        self.funcs.append(fi)

    def visit_AsyncFunctionDef(self, node):
        self.visit_FunctionDef(node)

    # branches
    def visit_If(self, node):        self.current().branches.append(node.lineno); self.generic_visit(node)
    def visit_While(self, node):     self.current().branches.append(node.lineno); self.generic_visit(node)
    def visit_For(self, node):       self.current().branches.append(node.lineno); self.generic_visit(node)
    def visit_AsyncFor(self, node):  self.current().branches.append(node.lineno); self.generic_visit(node)
    def visit_With(self, node):      self.current().branches.append(node.lineno); self.generic_visit(node)

    def visit_Try(self, node):
        self.current().branches.append(node.lineno)
        self.current().exceptions.append(("try", node.lineno, {}))
        self.generic_visit(node)

    def visit_ExceptHandler(self, node):
        hint = {}
        if node.type is not None:
            typ = try_unparse(node.type)
            if not typ and hasattr(node.type, "id"):
                typ = node.type.id
            if typ: hint["exception_type"] = typ
        self.current().branches.append(node.lineno)
        self.current().exceptions.append(("except", node.lineno, hint))
        self.generic_visit(node)

    def visit_Raise(self, node):
        hint = {}
        if node.exc is not None:
            text = try_unparse(node.exc)
            if text: hint["expr"] = text
            if isinstance(node.exc, ast.Call):
                fn = node.exc.func
                if isinstance(fn, ast.Name):
                    hint["exception_type"] = fn.id
                elif isinstance(fn, ast.Attribute):
                    hint["exception_type"] = fn.attr
                if node.exc.args:
                    a0 = node.exc.args[0]
                    if isinstance(a0, ast.Constant) and isinstance(a0.value, str):
                        hint["message_contains"] = a0.value[:80]
        self.current().exceptions.append(("raise", node.lineno, hint))
        self.generic_visit(node)

    def visit_Assert(self, node):
        hint = {}
        if node.msg and isinstance(node.msg, ast.Constant) and isinstance(node.msg.value, str):
            hint["message_contains"] = node.msg.value[:80]
        self.current().exceptions.append(("assert", node.lineno, hint))
        self.generic_visit(node)

    def visit_Name(self, node):
        if isinstance(node.ctx, ast.Store):
            self.current().add_def(node.id, node.lineno)
        elif isinstance(node.ctx, ast.Load):
            self.current().add_use(node.id, node.lineno)
        self.generic_visit(node)

    # dotted name
    def _dotted_name(self, node):
        if isinstance(node, ast.Name):
            nm = node.id
            if nm in self.alias_to_module:
                return self.alias_to_module[nm]
            if nm in self.symbol_to_module:
                return f"{self.symbol_to_module[nm]}.{nm}"
            return nm
        if isinstance(node, ast.Attribute):
            base = self._dotted_name(node.value)
            if base: return f"{base}.{node.attr}"
        return None

    # calls
    def visit_Call(self, node):
        qn = self._dotted_name(node.func) or ""
        if qn in ("open", "builtins.open"):
            self.current().side_effect_calls.append(("io_open", node.lineno))
        if any(qn.startswith(m + ".") for m in MOD_REQS):
            self.current().side_effect_calls.append(("net_requests", node.lineno))
        if any(qn.startswith(m + ".") for m in MOD_HTTPX):
            self.current().side_effect_calls.append(("net_httpx", node.lineno))
        if any(qn.startswith(pref) for pref in TEMPORAL_PREFIXES) \
           or any(qn.startswith(m + ".") for m in MOD_TEMPORAL):
            self.current().side_effect_calls.append(("temporal", node.lineno))
        if any(qn.startswith(m + ".") for m in MOD_SUBPROCESS):
            self.current().side_effect_calls.append(("subprocess", node.lineno))
        self.generic_visit(node)


# ===================== 대상 파일 =====================
candidate_files = sorted(set([*uncovered_map.keys(), *observed_outcomes.keys()]))
candidate_files = [fp for fp in candidate_files if is_source(fp)]


# ---------------- 가중치/하이퍼파라미터 ----------------
ALPHA_BRANCH = 0.45
BETA_DU      = 0.25
GAMMA_EXC    = 0.30
LAMBDA_1_CONST   = 0.20
LAMBDA_2_CONTEXT = 0.60
LAMBDA_3_TARGETS = 0.20
TOP_K = 10
OVERLAP_THETA = 0.50

assert abs((ALPHA_BRANCH + BETA_DU + GAMMA_EXC) - 1.0) < 1e-6
assert abs((LAMBDA_1_CONST + LAMBDA_2_CONTEXT + LAMBDA_3_TARGETS) - 1.0) < 1e-6


def coverage_gain_structural(b_cnt, du_cnt, exc_cnt):
    return ALPHA_BRANCH*b_cnt + BETA_DU*du_cnt + GAMMA_EXC*exc_cnt

def coverage_gain_total(target_lines, b_cnt, du_cnt, exc_cnt):
    return len(target_lines) + coverage_gain_structural(b_cnt, du_cnt, exc_cnt)

def generation_cost(context_size, target_count):
    return (LAMBDA_1_CONST*1.0) + (LAMBDA_2_CONTEXT*context_size) + (LAMBDA_3_TARGETS*target_count)


# ===================== AST → file_infos =====================
file_infos = {}
MAX_NEAR_GAP = 8

for file_abs in candidate_files:
    try:
        source = Path(file_abs).read_text(encoding="utf-8")
        tree = ast.parse(source)
    except Exception as e:
        print(f"⚠️ AST parse 실패: {file_abs}, {e}")
        continue

    rel = rel_from_proj(file_abs)
    visitor = ASTVisitor(rel)
    visitor.visit(tree)

    miss = set(uncovered_map.get(file_abs, []))
    half = set(half_hit_map.get(file_abs, []))

    for fi in visitor.funcs:
        tb = [ln for ln in fi.branches if (ln in miss or ln in half)]
        tu_pairs = []
        for var, defs in fi.defs.items():
            uses = sorted(fi.uses.get(var, []))
            defs_sorted = sorted(defs)
            for u in uses:
                if u not in miss:
                    continue
                d_le = [d for d in defs_sorted if d <= u]
                if not d_le:
                    continue
                d = max(d_le)
                tu_pairs.append((var, d, u))

        te = [(kind, line, hint)
              for (kind, line, hint) in fi.exceptions if line in miss]

        ctx = len({ln for ls in fi.defs.values() for ln in ls} |
                  {ln for ls in fi.uses.values() for ln in ls})

        if tb or tu_pairs or te:
            file_infos[(rel, fi.name)] = {
                "branches": sorted(tb),
                "def_uses": sorted(tu_pairs, key=lambda x: (x[2], x[1], x[0])),
                "exceptions": sorted(te, key=lambda x: (x[1], x[0])),
                "func_lineno": fi.lineno,
                "context_size": ctx,
            }


# ===================== 후보 goal 생성 =====================
def make_goal(file_rel, func, func_lineno, branches, def_uses, exceptions, context_size):
    target_lines = set(branches)
    for _, d, u in def_uses: target_lines.update([d, u])
    for _, line, _ in exceptions: target_lines.add(line)

    b_cnt = len(branches)
    du_cnt = len(def_uses)
    exc_cnt = len(exceptions)

    gain = coverage_gain_total(target_lines, b_cnt, du_cnt, exc_cnt)
    cost = generation_cost(context_size, len(target_lines))

    return {
        "file": file_rel,
        "function": {"name": func, "lineno": func_lineno},
        "components": {
            "branches": [{"line": b} for b in branches],
            "def_uses": [{"var": v, "def_line": d, "use_line": u} for (v, d, u) in def_uses],
            "exceptions": [{"kind": k, "line": ln, **hint} for (k, ln, hint) in exceptions],
        },
        "target_lines": sorted(target_lines),
        "coverage_gain": float(gain),
        "generation_cost": float(cost),
        "context_size": context_size,
    }


candidates = []

for (file_rel, func), info in file_infos.items():
    tb = info["branches"]
    tu = info["def_uses"]
    te = info["exceptions"]
    ctx = info["context_size"]
    func_lineno = info["func_lineno"]

    # 단일 요소
    for b in tb:              candidates.append(make_goal(file_rel, func, func_lineno, [b], [], [], ctx))
    for (v, d, u) in tu:      candidates.append(make_goal(file_rel, func, func_lineno, [], [(v, d, u)], [], ctx))
    for (k, ln, h) in te:     candidates.append(make_goal(file_rel, func, func_lineno, [], [], [(k, ln, h)], ctx))

    # 근접 조합
    for b in tb:
        for (v, d, u) in tu:
            if max(b, u) - min(b, d) <= MAX_NEAR_GAP:
                candidates.append(make_goal(file_rel, func, func_lineno, [b], [(v, d, u)], [], ctx))

    for b in tb:
        for (k, ln, h) in te:
            if abs(b - ln) <= MAX_NEAR_GAP:
                candidates.append(make_goal(file_rel, func, func_lineno, [b], [], [(k, ln, h)], ctx))

    for (v, d, u) in tu:
        for (k, ln, h) in te:
            if max(u, ln) - min(d, ln) <= MAX_NEAR_GAP:
                candidates.append(make_goal(file_rel, func, func_lineno, [], [(v, d, u)], [(k, ln, h)], ctx))

    for b in tb:
        for (v, d, u) in tu:
            for (k, ln, h) in te:
                lines = [b, d, u, ln]
                if max(lines) - min(lines) <= MAX_NEAR_GAP:
                    candidates.append(make_goal(file_rel, func, func_lineno,
                                                [b], [(v, d, u)], [(k, ln, h)], ctx))


# ===================== Algorithm 1 (19줄) 구현 =====================

def overlap_basic(g, h):
    tg = set(g["target_lines"])
    th = set(h["target_lines"])
    if not tg:
        return 0.0
    return len(tg & th) / len(tg)

# goc 계산
a = 0.7
for g in candidates:
    gain = g["coverage_gain"]
    cost = g["generation_cost"]
    g["goc"] = a * gain + (1 - a) * (1 / (cost + 1))

# ---- filtering (line 12~16) ----
filtered = []
for g in candidates:
    if g["goc"] <= 0:
        continue
    bad = False
    for h in filtered:
        if overlap_basic(g, h) > OVERLAP_THETA and h["goc"] >= g["goc"]:
            bad = True
            break
    if not bad:
        filtered.append(g)

# ---- 정렬 (line 17) ----
filtered_sorted = sorted(filtered, key=lambda x: x["goc"], reverse=True)

# ---- top-k 선택 (line 18) ----
selected = filtered_sorted[:TOP_K]

# ---- 정제 및 저장 ----
for i, g in enumerate(selected, 1):
    g["id"] = f"{i:04d}"
    g["coverage_gain"] = round(g["coverage_gain"], 6)
    g["generation_cost"] = round(g["generation_cost"], 6)
    g["score"] = round(g["goc"], 6)

(ART_DIR / "goals_raw.json").write_text(json.dumps(selected, indent=2, ensure_ascii=False), encoding="utf-8")

ranked = [
    {
        "id": g["id"],
        "file": g["file"],
        "function": g["function"],
        "components": g["components"],
        "target_lines": g["target_lines"],
        "coverage_gain": g["coverage_gain"],
        "generation_cost": g["generation_cost"],
        "score": g["score"],
        "context_size": g["context_size"],
    }
    for g in selected
]

(ART_DIR / "goals_ranked.json").write_text(json.dumps(ranked, indent=2, ensure_ascii=False), encoding="utf-8")

print(f"✅ 복합 목표 생성 완료 (Algorithm1, top-k={TOP_K})")
print(f" - 원본 후보 수: {len(candidates)}")
print(f" - filtered 후보 수: {len(filtered)}")
print(f" - 최종 선택된 목표 수: {len(selected)}")
print(" - 저장: goals_raw.json, goals_ranked.json")


✅ 복합 목표 생성 완료 (Algorithm1, top-k=10)
 - 원본 후보 수: 94
 - filtered 후보 수: 50
 - 최종 선택된 목표 수: 10
 - 저장: goals_raw.json, goals_ranked.json


In [5]:
#@title 3-2c) 입력 제약 자동 추출기

import ast
import json
from pathlib import Path

PROJ_PATH = Path(PROJ).resolve()
ART_DIR = PROJ_PATH / "run_artifacts" / "run1"

GOALS_RANKED = ART_DIR / "goals_ranked.json"
OUT_FILE = ART_DIR / "goals_enriched.json"

assert GOALS_RANKED.exists(), "goals_ranked.json 없어요. 3-2b 먼저 실행하세요."

goals = json.loads(GOALS_RANKED.read_text(encoding="utf-8"))

# ----------------------------------------------------------------------
# 공용 유틸
# ----------------------------------------------------------------------

def load_ast(path: Path):
    try:
        return ast.parse(path.read_text(encoding="utf-8"))
    except Exception:
        return None

def annotate_parent(root):
    for node in ast.walk(root):
        for child in ast.iter_child_nodes(node):
            setattr(child, "parent", node)

def u(node):
    try:
        return ast.unparse(node)
    except:
        return None

# ----------------------------------------------------------------------
# 타입/파라미터 제약
# ----------------------------------------------------------------------

def extract_param_constraints(func_node: ast.FunctionDef):
    cons = []
    args = func_node.args.args

    for a in args:
        name = a.arg
        if a.annotation:
            ann = u(a.annotation) or ""
            ann_l = ann.lower()
            if "int" in ann_l:
                cons.append(f"{name}: integer")
            elif "float" in ann_l:
                cons.append(f"{name}: float")
            elif "str" in ann_l:
                cons.append(f"{name}: string")
            elif "bool" in ann_l:
                cons.append(f"{name}: boolean")
            else:
                cons.append(f"{name}: type={ann}")

        # 기본값 기반 제약
        if func_node.args.defaults:
            idx = args.index(a) - (len(args) - len(func_node.args.defaults))
            if idx >= 0:
                dv = func_node.args.defaults[idx]
                cons.append(f"{name}: default={u(dv)}")

    return cons

# ----------------------------------------------------------------------
# 심볼릭 조건식(parser) → value-domain constraint
# ----------------------------------------------------------------------

def _translate_compare(node):
    """
    x > 0 → "x > 0"
    0 < x < 10 → "0 < x < 10"
    """
    try:
        return u(node)
    except:
        return None

def extract_branch_constraints(func_node: ast.FunctionDef):
    cons = []

    for node in ast.walk(func_node):

        # if 조건
        if isinstance(node, ast.If):
            cond = _translate_compare(node.test)
            if cond:
                cons.append(f"condition: {cond}")

        # assert 조건
        if isinstance(node, ast.Assert):
            cond = _translate_compare(node.test)
            if cond:
                cons.append(f"assert: {cond}")

        # 조건 안에 값 비교가 있으면 value-domain 추론
        if isinstance(node, ast.Compare):
            left = u(node.left)
            rights = [u(r) for r in node.comparators]
            ops = node.ops

            if left and all(rights):
                if isinstance(ops[0], ast.Gt):
                    cons.append(f"{left} > {rights[0]}")
                if isinstance(ops[0], ast.Lt):
                    cons.append(f"{left} < {rights[0]}")
                if isinstance(ops[0], ast.GtE):
                    cons.append(f"{left} >= {rights[0]}")
                if isinstance(ops[0], ast.LtE):
                    cons.append(f"{left} <= {rights[0]}")
                if isinstance(ops[0], ast.Eq):
                    cons.append(f"{left} == {rights[0]}")
                if isinstance(ops[0], ast.NotEq):
                    cons.append(f"{left} != {rights[0]}")

    return cons

# ----------------------------------------------------------------------
# def-use 기반 value-flow 제약
# ----------------------------------------------------------------------

def extract_def_use_constraints(func_node: ast.FunctionDef, goal):
    pairs = goal["components"]["def_uses"]
    cons = []

    for pair in pairs:
        var = pair["var"]
        d = pair["def_line"]
        u_line = pair["use_line"]
        cons.append(f"value-flow: {var} defined@{d} → used@{u_line}")

    return cons

# ----------------------------------------------------------------------
# raise/except 기반 예외 조건 추론
# ----------------------------------------------------------------------

def extract_exception_constraints(func_node: ast.FunctionDef):
    cons = []
    for node in ast.walk(func_node):
        if isinstance(node, ast.Raise):
            exc = u(node.exc)
            if exc:
                cons.append(f"exception: raises {exc}")

        if isinstance(node, ast.ExceptHandler):
            typ = u(node.type) if node.type else None
            if typ:
                cons.append(f"exception-handler: {typ}")

    return cons

# ----------------------------------------------------------------------
# 필드 접근 / subscript 기반 구조 제약
# ----------------------------------------------------------------------

def extract_field_constraints(func_node: ast.FunctionDef):
    cons = []
    for node in ast.walk(func_node):

        # obj.attr
        if isinstance(node, ast.Attribute):
            expr = u(node)
            if expr:
                cons.append(f"needs field: {expr}")

        # obj["key"]
        if isinstance(node, ast.Subscript):
            expr = u(node)
            if expr:
                cons.append(f"needs key: {expr}")

    return cons

# ----------------------------------------------------------------------
# relevance filtering: target_lines 근처의 제약만 남기기
# ----------------------------------------------------------------------

def filter_relevant(constraints, func_node, goal):
    target = set(goal["target_lines"])
    if not target:
        return constraints

    relevant = []

    for c in constraints:
        # 매우 단순하게: 숫자 라인이 포함되면 필터링
        found = False
        for t in target:
            if f"@{t}" in c or f"{t}" in c:
                found = True
                break
        if found:
            relevant.append(c)
        else:
            # branch/assert는 전체 함수에 영향 → keep
            if c.startswith("condition") or c.startswith("assert"):
                relevant.append(c)

    return list(dict.fromkeys(relevant))

# ----------------------------------------------------------------------
# 메인
# ----------------------------------------------------------------------

goals_out = []

for g in goals:
    file_rel = g["file"]
    fname = g["function"]["name"]

    src = (PROJ_PATH / file_rel).resolve()
    tree = load_ast(src)

    if tree is None:
        g["input_constraints"] = []
        goals_out.append(g)
        continue

    annotate_parent(tree)

    func_node = None
    for node in ast.walk(tree):
        if isinstance(node, ast.FunctionDef) and node.name == fname:
            func_node = node
            break

    if not func_node:
        g["input_constraints"] = []
        goals_out.append(g)
        continue

    # ---- 추론 모듈 합치기 ----
    c_param = extract_param_constraints(func_node)
    c_branch = extract_branch_constraints(func_node)
    c_du = extract_def_use_constraints(func_node, g)
    c_exc = extract_exception_constraints(func_node)
    c_field = extract_field_constraints(func_node)

    all_c = c_param + c_branch + c_du + c_exc + c_field
    all_c = list(dict.fromkeys(all_c))

    # relevance filter
    all_c = filter_relevant(all_c, func_node, g)

    g["input_constraints"] = all_c
    goals_out.append(g)

# ---------------- 저장 ----------------
OUT_FILE.write_text(
    json.dumps(goals_out, ensure_ascii=False, indent=2),
    encoding="utf-8"
)

print("✅ 심층 입력 제약 추출 완료 →", OUT_FILE)
print("총 목표:", len(goals_out))


✅ 심층 입력 제약 추출 완료 → /content/money-transfer-project-template-python/run_artifacts/run1/goals_enriched.json
총 목표: 10


In [6]:
#@title 3-3) LLM 프롬프트 생성
import json, re
from pathlib import Path

PROJ_PATH = Path(PROJ).resolve()
ART_DIR = PROJ_PATH / "run_artifacts" / "run1"

GOALS_RANKED = ART_DIR / "goals_ranked.json"
GOALS_ENRICHED = ART_DIR / "goals_enriched.json"
LLM_OUT = ART_DIR / "llm_prompts.jsonl"

assert GOALS_RANKED.exists(), "goals_ranked.json 없음 (3-2b 먼저 실행)"
assert GOALS_ENRICHED.exists(), "goals_enriched.json 없음 (3-2c 먼저 실행)"

ranked = json.loads(GOALS_RANKED.read_text(encoding="utf-8"))
enriched = json.loads(GOALS_ENRICHED.read_text(encoding="utf-8"))

# --- enriched 를 id 기준으로 Lookup
E = {g["id"]: g for g in enriched}

def to_mod(file):
    s = file.replace("\\", "/")
    if s.startswith("src/"): s = s[4:]
    if s.endswith(".py"): s = s[:-3]
    return s.replace("/", ".")

def suggest(goal):
    mod = to_mod(goal["file"]).split(".")[-1]
    func = goal["function"]["name"]
    safe = lambda x: re.sub(r"[^a-zA-Z0-9_]+", "_", str(x))
    return f"test_gen_{safe(goal['id'])}_{safe(mod)}_{safe(func)}.py"

# --- 시스템 메시지 (강화)
SYSTEM = (
"역할: 당신은 주어진 실행 목표(분기/정의-사용/예외)를 타격하는 pytest 테스트 코드를 생성하는 엔진입니다.\n"
"출력: 오직 하나의 JSON 객체만 반환하고 주석/markdown 금지.\n"
"{\n"
'  "filename": "test_*.py",\n'
'  "tests": [ {"name": "test_*", "code": "<pytest code>"} ]\n'
"}\n"
"\n"
"엄격 규칙:\n"
"• importlib.import_module + getattr 로만 심볼 접근 (없을 때만 guarded skip 허용)\n"
"• 모듈 레벨 skip 금지. skip은 테스트 함수 내부 + 심볼 없음 상황에서만.\n"
"• 최소 1개 assert 또는 pytest.raises 필수.\n"
"• 분기는 양 경로를 별도 테스트로 수행(half-hit 시 2개 필요).\n"
"• def-use 경로는 실제 상태/반환에 반영되도록 assert.\n"
"• 예외는 pytest.raises로 타입(+가능하면 메시지) 검증.\n"
"• IO/시간/env/net/Temporal 등 외부는 모두 monkeypatch.\n"
"• async 대상은 반드시 async 테스트 + await. asyncio.run 금지.\n"
"• Temporal은 Client/Worker/Workflow 등 공식 엔트리포인트만 더미 패치.\n"
)

def priority(g):
    return float(g.get("score", g.get("coverage_gain", 0.0)))

ranked_sorted = sorted(ranked, key=lambda x: -priority(x))

# --- 파일 생성 ---
with LLM_OUT.open("w", encoding="utf-8") as outf:
    for rank, g in enumerate(ranked_sorted, start=1):

        e = E[g["id"]]
        mod = to_mod(g["file"])
        filename = suggest(g)

        # --- 3-2c 입력 제약 반영
        input_constraints = e.get("input_constraints", [])

        # --- 테스트 입력값 생성 규칙 (target-line 히트용)
        hit_recipe = [
            "• target_lines와 직접 연결된 조건식/비교식을 충족하는 입력값을 구성할 것.",
            "• 조건식(예: amount > 0, 0 < x < 10)을 만족하는 최소/경계값 우선 생성.",
            "• def-use 흐름(v defined@d → used@u)은 use-line에서 값이 실제 반영되도록 입력 구성.",
            "• needs field: obj.x / needs key: data['x'] 가 있는 경우 해당 구조 가진 dict/object 제공.",
            "• exception 목표가 있을 경우 raise 발생 조건을 정확히 충족시키는 입력 구성.",
        ]

        USER = {
            "schema_version": "v2",
            "identifier": {
                "id": g["id"],
                "rank": rank,
                "priority": priority(g),
                "basis": "score" if "score" in g else "coverage_gain"
            },
            "project": {
                "root": str(PROJ_PATH),
                "module": mod
            },
            "goal": {
                "file": g["file"],
                "function": g["function"],
                "components": g["components"],
                "target_lines": g["target_lines"],
                "input_constraints": input_constraints,
            },
            "input_synthesis_recipe": hit_recipe,
            "constraints": {
                "filename_suggestion": filename,
                "import_policy": {
                    "strategy": "importlib_only",
                    "on_missing": "pytest.skip"
                },
                "execution_contract": {
                    "must_hit_at_least_n_target_lines": 1,
                    "require_two_tests_for_half_hit": bool(g["components"]["branches"]),
                    "test_name_must_include_hit_lines": True
                },
                "isolation_policy": {
                    "no_fs_no_net": True,
                    "patch_time": True,
                    "patch_env": True,
                    "forbid_asyncio_run": True,
                    "forbid_temporal_real_runs": True
                },
                "assert_policy": {
                    "prefer_pytest_raises": True,
                    "prefer_explicit_asserts": True,
                    "require_state_assert": True,
                    "no_unrelated_asserts": True
                },
                "async_policy": {
                    "mark_asyncio_and_await_coroutines": True,
                    "require_async_test_if_coroutine": True,
                    "forbid_sync_call_of_coroutines": True,
                    "forbid_asyncio_run": True
                },
                "temporal_policy": {
                    "patch_entrypoints_only": True,
                    "require_temporal_mocks": True,
                    "preferred_targets": [
                        "temporalio.client.Client",
                        "temporalio.worker.Worker",
                        "temporalio.workflow.Workflow"
                    ]
                }
            },
            "instructions": [
                "JSON 형태만 반환할 것.",
                "입력 제약 및 branch/def-use/exceptions를 기반으로 target_lines를 반드시 타격할 것.",
                "value-range, 조건식, 구조적 제약을 충족하는 입력값을 직접 구성할 것.",
                "async/Temporal/test isolation 규칙 준수."
            ]
        }

        rec = {
            "meta": {
                "id": g["id"],
                "rank": rank,
                "priority": priority(g),
                "file": g["file"],
                "function": g["function"]["name"],
                "coverage_gain": g["coverage_gain"],
                "score": g["score"],
                "suggested_filename": filename,
                "input_constraints_count": len(input_constraints),
            },
            "messages": [
                {"role": "system", "content": SYSTEM},
                {"role": "user", "content": json.dumps(USER, ensure_ascii=False, indent=2)}
            ]
        }

        outf.write(json.dumps(rec, ensure_ascii=False) + "\n")

print("✅ LLM 프롬프트 생성 완료 →", LLM_OUT)
print("총 목표:", len(ranked_sorted))
print("예시 파일명:", suggest(ranked_sorted[0]))


✅ LLM 프롬프트 생성 완료 → /content/money-transfer-project-template-python/run_artifacts/run1/llm_prompts.jsonl
총 목표: 10
예시 파일명: test_gen_0001_activities_refund.py


In [7]:
#@title 3-4) 테스트 코드 생성
import os
import re
import json
import time
import ast
from pathlib import Path
import httpx
import backoff
from openai import OpenAI, APIError, RateLimitError, APIConnectionError

# ---------- 경로 설정 ----------
ART_DIR = Path(PROJ) / "run_artifacts" / "run1"
LLM_PROMPTS_PATH = ART_DIR / "llm_prompts.jsonl"
GEN_DIR = Path(PROJ) / "generated_tests"
RAW_DIR = ART_DIR / "_raw"
ERR_DIR = ART_DIR / "_errors"
GEN_DIR.mkdir(parents=True, exist_ok=True)
RAW_DIR.mkdir(parents=True, exist_ok=True)
ERR_DIR.mkdir(parents=True, exist_ok=True)

# ---------- OpenAI 클라이언트 ----------
if not os.getenv("OPENAI_API_KEY"):
    raise RuntimeError("OPENAI_API_KEY가 설정되지 않았습니다. 3-0 단계에서 .env를 로드했는지 확인하세요.")

http_client = httpx.Client(
    timeout=180.0,
    follow_redirects=True,
    limits=httpx.Limits(max_connections=1, max_keepalive_connections=0),
    transport=httpx.HTTPTransport(retries=5),
)
client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    base_url="https://api.openai.com/v1",
    http_client=http_client,
)

# ---------- 유틸 ----------
_slug_re = re.compile(r"[^a-z0-9_]+")
def slugify(s: str, maxlen: int = 40) -> str:
    s = s.lower().strip().replace("-", "_").replace(" ", "_")
    s = _slug_re.sub("_", s)
    s = re.sub(r"_+", "_", s).strip("_")
    return s[:maxlen] or "t"

def strip_fences(s: str) -> str:
    s = re.sub(r"^```[a-zA-Z0-9]*\s*", "", s.strip())
    s = re.sub(r"\s*```$", "", s)
    return s

def ensure_unique_path(base: Path) -> Path:
    p = base
    i = 2
    while p.exists():
        p = base.with_name(f"{base.stem}_{i}{base.suffix}")
        i += 1
    return p

def write_error(goal_id: str, kind: str, payload: dict, idx: int | None = None):
    tag = f"goal_{goal_id}_{kind}" if idx is None else f"goal_{goal_id}_t{idx}_{kind}"
    (ERR_DIR / f"{tag}.json").write_text(
        json.dumps(payload, ensure_ascii=False, indent=2),
        encoding="utf-8"
    )

# ---------- PATCH (1): 보정기 - 필수 import 자동 주입 강화 ----------
def auto_fix_imports(code: str) -> str:
    """
    기존 기능 + 강화된 import 자동 삽입
    - pytest
    - importlib
    - SimpleNamespace
    - asyncio
    """
    lines = code.splitlines()
    text = "\n".join(lines)

    need_pytest = ("pytest." in text) and not re.search(r"^\s*import\s+pytest\b", text, re.M)
    need_importlib = ("importlib." in text) and not re.search(r"^\s*import\s+importlib\b", text, re.M)
    need_simple = ("SimpleNamespace" in text) and not re.search(r"from\s+types\s+import\s+SimpleNamespace", text, re.M)
    need_asyncio = ("asyncio." in text) and not re.search(r"^\s*import\s+asyncio\b", text, re.M)

    prepend = []
    if need_pytest: prepend.append("import pytest")
    if need_importlib: prepend.append("import importlib")
    if need_simple: prepend.append("from types import SimpleNamespace")
    if need_asyncio: prepend.append("import asyncio")

    if not prepend:
        return code

    # from __future__ import ... 뒤에 넣기
    new_lines = []
    inserted = False
    for ln in lines:
        new_lines.append(ln)
        if (not inserted) and not ln.strip().startswith("from __future__"):
            new_lines = prepend + new_lines
            inserted = True
            break

    if not inserted:
        new_lines = prepend + lines

    return "\n".join(new_lines) + ("\n" if not new_lines[-1].endswith("\n") else "")

# ---------- 보정기: runner 제거 ----------
def sanitize_test_code(code: str) -> str:
    patterns = [
        re.compile(r"(?ms)^\s*if\s+__name__\s*==\s*['\"]__main__['\"]\s*:\s*\n(?:\s+.*\n?)+$"),
        re.compile(r"(?m)^\s*pytest\.main\s*\(.*?\)\s*$"),
        re.compile(r"(?m)^\s*unittest\.main\s*\(.*?\)\s*$"),
    ]
    new = code
    for pat in patterns:
        new = pat.sub("", new)
    return new.strip() + "\n"

# ---------- 검증 로직 ----------
RE_IMPORTLIB = re.compile(r"\bimportlib\.import_module\s*\(")
RE_PYTEST_RAISES = re.compile(r"\bpytest\.raises\s*\(")

def validate_json_schema(result: dict) -> tuple[bool, list[str]]:
    reasons = []
    if not isinstance(result, dict):
        return False, ["not_a_json_object"]
    if "tests" not in result:
        reasons.append("missing_tests")
    else:
        if not isinstance(result["tests"], list) or len(result["tests"]) == 0:
            reasons.append("tests_empty_or_not_list")
        else:
            for i, t in enumerate(result["tests"], start=1):
                if not isinstance(t, dict):
                    reasons.append(f"test_{i}_not_object"); continue
                if "code" not in t:
                    reasons.append(f"test_{i}_missing_code")
                if "name" not in t:
                    reasons.append(f"test_{i}_missing_name")
    if "filename" in result:
        fn = str(result["filename"])
        if not fn.endswith(".py"):
            reasons.append("filename_not_py")
    return (len(reasons) == 0), reasons

def parse_ast_or_error(code: str):
    try:
        return ast.parse(code), None
    except SyntaxError as e:
        return None, f"syntax_error:{e.msg}@L{e.lineno}"

def extract_test_funcs(tree: ast.AST) -> list[ast.FunctionDef]:
    return [n for n in ast.walk(tree)
            if isinstance(n, ast.FunctionDef) and n.name.startswith("test_")]

def has_assert_or_raises(tree: ast.AST, code: str) -> bool:
    has_assert_stmt = any(isinstance(n, ast.Assert) for n in ast.walk(tree))
    has_pytest_raises = bool(RE_PYTEST_RAISES.search(code))
    return has_assert_stmt or has_pytest_raises

def uses_importlib(code: str) -> bool:
    return bool(RE_IMPORTLIB.search(code))

# ---------- skip 가드 ----------
def _parent_map(tree: ast.AST):
    parent = {}
    for node in ast.walk(tree):
        for child in ast.iter_child_nodes(node):
            parent[child] = node
    return parent

def _is_guarded_skip(call: ast.Call, parent_map, src: str) -> bool:
    cur = call
    while cur in parent_map:
        cur = parent_map[cur]
        if isinstance(cur, ast.If):
            test = cur.test
            if (
                isinstance(test, ast.Compare)
                and len(test.ops) == 1
                and isinstance(test.ops[0], ast.Is)
                and len(test.comparators) == 1
                and isinstance(test.comparators[0], ast.Constant)
                and test.comparators[0].value is None
            ):
                return True
        if isinstance(cur, ast.ExceptHandler):
            t = cur.type
            if isinstance(t, ast.Name) and t.id in {"ImportError", "NameError"}:
                return True
    return False

def has_unconditional_skip(code: str) -> tuple[bool, list[int]]:
    try:
        tree = ast.parse(code)
    except SyntaxError:
        return (False, [])
    parent = _parent_map(tree)
    bad_lines = []
    for node in ast.walk(tree):
        if isinstance(node, ast.Call):
            f = node.func
            if (
                isinstance(f, ast.Attribute)
                and isinstance(f.value, ast.Name)
                and f.value.id == "pytest"
                and f.attr == "skip"
            ):
                if not _is_guarded_skip(node, parent, code):
                    bad_lines.append(getattr(node, "lineno", -1))
    return (len(bad_lines) > 0, bad_lines)

# ---------- PATCH (2): 최소 실행 요건 완화 ----------
def minimal_viability_checks(code: str) -> tuple[bool, list[str], dict]:
    reasons = []
    meta = {"warnings": []}

    # 아주 짧은 코드 경고만
    if len(code.strip()) < 50:
        meta["warnings"].append("very_short_code")

    tree, synerr = parse_ast_or_error(code)
    if synerr:
        reasons.append(synerr)
        return False, reasons, meta

    tests = extract_test_funcs(tree)
    if not tests:
        reasons.append("no_test_functions")

    if not has_assert_or_raises(tree, code):
        reasons.append("no_assert_or_raises")

    # importlib 없는 경우 경고만
    if not uses_importlib(code):
        meta["warnings"].append("no_importlib_import_module")

    # forbidden import from direct absolute usage (새로운 fail 조건)
    if re.search(r"from\s+[\w\.]+\s+import\s+\w+", code):
        meta["warnings"].append("direct_from_import_detected")

    has_bad_skip, bad_lines = has_unconditional_skip(code)
    if has_bad_skip:
        reasons.append(f"unconditional_skip_detected@{bad_lines}")

    # runner 금지
    if re.search(r"pytest\.main\s*\(", code):
        reasons.append("forbidden_runner_invocation:pytest.main")
    if re.search(r"unittest\.main\s*\(", code):
        reasons.append("forbidden_runner_invocation:unittest.main")

    return (len(reasons) == 0), reasons, meta

# ---------- OpenAI ----------
@backoff.on_exception(
    backoff.expo,
    (APIConnectionError, APIError, RateLimitError),
    max_tries=8,
    max_time=300
)
def call_openai_with_retry(messages):
    resp = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        response_format={"type": "json_object"},
        timeout=180.0,
    )
    return resp.choices[0].message.content

# ---------- PATCH (3): filename 정책 강화 ----------
def choose_filename(base_name: str | None, goal_id: str, idx: int, tests_len: int):
    """
    새로운 filename 정책:
    - result.filename 있으면 그대로 우선 사용
    - 단, 다중 테스트인 경우 suffix 붙임
    """
    if base_name:
        base = Path(base_name).stem
        if tests_len > 1:
            return f"{base}_{idx}.py"
        else:
            return f"{base}.py"
    else:
        slug = slugify(f"goal_{goal_id}")
        if tests_len > 1:
            return f"test_{goal_id}_{slug}_{idx}.py"
        return f"test_{goal_id}_{slug}.py"


# ---------- 메인 루프 ----------
gen_log_path = ART_DIR / "gen_log.jsonl"
ok_count = 0
fail_count = 0

with LLM_PROMPTS_PATH.open("r", encoding="utf-8") as f_in, gen_log_path.open("w", encoding="utf-8") as f_log:
    for line in f_in:
        rec = json.loads(line)
        goal_id = rec["meta"]["id"]
        messages = rec["messages"]
        print(f"\n🚀 Goal {goal_id} 테스트 생성 요청…")

        try:
            out_text = call_openai_with_retry(messages)
        except Exception as e:
            fail_count += 1
            write_error(goal_id, "request_error", {"error": str(e)})
            print(f"❌ Goal {goal_id} 요청 실패: {e}")
            continue

        (RAW_DIR / f"goal_{goal_id}_raw.json").write_text(out_text, encoding="utf-8")

        try:
            cleaned = strip_fences(out_text)
            result = json.loads(cleaned)
        except Exception as e:
            fail_count += 1
            write_error(goal_id, "json_parse_error", {"error": str(e), "raw": out_text[:2000]})
            print(f"❌ Goal {goal_id} JSON 파싱 실패: {e}")
            continue

        ok_schema, schema_reasons = validate_json_schema(result)
        if not ok_schema:
            fail_count += 1
            write_error(goal_id, "schema_error", {"reasons": schema_reasons, "result": result})
            print(f"❌ Goal {goal_id} 스키마 오류: {schema_reasons}")
            continue

        tests = result.get("tests", [])
        base_name = result.get("filename")

        saved_files = []
        excluded = []

        for idx, t in enumerate(tests, start=1):
            code = strip_fences(t.get("code", ""))
            code = sanitize_test_code(code)
            code = auto_fix_imports(code)

            ok_min, reasons, meta = minimal_viability_checks(code)
            if not ok_min:
                excluded.append({"index": idx, "name": t.get("name"), "reasons": reasons, **meta})
                write_error(goal_id, "min_viability", {"index": idx, "name": t.get("name"), "reasons": reasons, **meta}, idx)
                print(f"⚠️ Goal {goal_id} 테스트 #{idx} 제외: {reasons}")
                continue

            out_filename = choose_filename(base_name, goal_id, idx, len(tests))
            out_path = ensure_unique_path(GEN_DIR / out_filename)
            out_path.write_text(code, encoding="utf-8")
            saved_files.append(out_path.name)
            print(f"✅ 저장: {out_path.name} (warnings: {','.join(meta.get('warnings', [])) or '없음'})")

        if saved_files:
            ok_count += 1
            f_log.write(json.dumps({"goal_id": goal_id, "saved_files": saved_files, "excluded_tests": excluded}, ensure_ascii=False) + "\n")
        else:
            fail_count += 1
            write_error(goal_id, "no_valid_tests", {"result_head": result, "excluded_tests": excluded})
            print(f"❌ Goal {goal_id} 유효 테스트 없음 → 기록만 남김")

print(f"\n✅ 생성 단계 종료: 성공 {ok_count} / 실패 {fail_count}")
print(f"   • 저장 폴더 : {GEN_DIR}")
print(f"   • 원문 보관 : {RAW_DIR}")
print(f"   • 에러/제외 : {ERR_DIR}")
print(f"   • 로그 파일 : {gen_log_path}")



🚀 Goal 0001 테스트 생성 요청…
✅ 저장: test_gen_0001_activities_refund_1.py (warnings: 없음)
✅ 저장: test_gen_0001_activities_refund_2.py (warnings: 없음)

🚀 Goal 0002 테스트 생성 요청…
✅ 저장: test_gen_0002_activities_refund_1.py (warnings: 없음)
✅ 저장: test_gen_0002_activities_refund_2.py (warnings: 없음)

🚀 Goal 0003 테스트 생성 요청…
✅ 저장: test_gen_0003_activities_refund_1.py (warnings: 없음)
✅ 저장: test_gen_0003_activities_refund_2.py (warnings: 없음)

🚀 Goal 0004 테스트 생성 요청…
⚠️ Goal 0004 테스트 #1 제외: ['no_test_functions', 'unconditional_skip_detected@[12]']
⚠️ Goal 0004 테스트 #2 제외: ['no_test_functions', 'unconditional_skip_detected@[12]']
❌ Goal 0004 유효 테스트 없음 → 기록만 남김

🚀 Goal 0005 테스트 생성 요청…
⚠️ Goal 0005 테스트 #1 제외: ['no_test_functions', 'unconditional_skip_detected@[12]']
⚠️ Goal 0005 테스트 #2 제외: ['no_test_functions', 'unconditional_skip_detected@[12]']
❌ Goal 0005 유효 테스트 없음 → 기록만 남김

🚀 Goal 0006 테스트 생성 요청…
⚠️ Goal 0006 테스트 #1 제외: ['unconditional_skip_detected@[9]']
⚠️ Goal 0006 테스트 #2 제외: ['unconditional_skip_detected@[9]'

In [8]:
#@title 3-5) 기존 tests(프루닝) + 생성 tests 격리 실행 · 로그 수집 · 샤드 결합 · 향상치 계산 (autosetup 포함)
import os, sys, json, re, time, subprocess, shutil, shlex
from pathlib import Path
from datetime import datetime, timezone
from lxml import etree

# ==== 경로/상수 ====
assert 'PROJ' in globals(), "3-0 단계를 먼저 실행하세요."
PROJ = Path(PROJ).resolve()
ART_DIR = PROJ / "run_artifacts" / "run1"
GEN_DIR = PROJ / "generated_tests"
TESTS_DIR = PROJ / "tests"                   # 원본(백업용)
PRUNED_DIR = PROJ / "Pruned_Base_Tests"      # 프루닝 집합(우선)
LOG_DIR = ART_DIR / "logs"
COV_SHARDS_DIR = ART_DIR / "cov_shards"
HTML_DIR_GEN = PROJ / "htmlcov_gen"

ART_DIR.mkdir(parents=True, exist_ok=True)
LOG_DIR.mkdir(parents=True, exist_ok=True)
COV_SHARDS_DIR.mkdir(parents=True, exist_ok=True)
HTML_DIR_GEN.mkdir(parents=True, exist_ok=True)
GEN_DIR.mkdir(parents=True, exist_ok=True)

RCFILE = PROJ / ".coveragerc"
rc_opt = f" --rcfile {RCFILE}" if RCFILE.exists() else ""

# 실행 파라미터
PY_EXE = sys.executable
TIMEOUT_SEC_GEN = 45            # 생성 테스트 파일 1개당 타임아웃 (늘림)
TIMEOUT_SEC_BASE = 45           # (프루닝된) 기존 테스트 파일 1개당 타임아웃
PYTEST_FLAGS = "-q -s"
ENV_BASE = os.environ.copy()

# goal_id 추출
RE_GOAL = re.compile(r"(?:^|[_-])(?P<gid>\d{4})(?:[_-]|$)")

# ==== Autosetup: pytest.ini + generated_tests/conftest.py ====
def ensure_pytest_ini():
    ini = PROJ / "pytest.ini"
    base = []
    if ini.exists():
        base = ini.read_text(encoding="utf-8").splitlines()

    def set_or_append(lines, key, value):
        if not any(l.strip().startswith("[pytest]") for l in lines):
            lines = ["[pytest]"] + lines
        found = False
        for i, l in enumerate(lines):
            if l.strip().startswith(key):
                lines[i] = f"{key} = {value}"
                found = True
                break
        if not found:
            try:
                idx = next(i for i,l in enumerate(lines) if l.strip().startswith("[pytest]"))
            except StopIteration:
                idx = -1
            insert_at = idx+1 if idx>=0 else len(lines)
            lines.insert(insert_at, f"{key} = {value}")
        return lines

    lines = base[:]
    lines = set_or_append(lines, "asyncio_mode", "auto")  # pytest-asyncio 자동 모드
    if not any(l.strip().startswith("addopts") for l in lines):
        lines.append("addopts = -q -s")
    ini.write_text("\n".join(lines) + "\n", encoding="utf-8")
    print("✅ pytest.ini ensured →", ini)

def ensure_conftest_autouse():
    cf = GEN_DIR / "conftest.py"
    payload = (
        "import os, asyncio, pytest, importlib, inspect\n"
        "\n"
        "@pytest.fixture(autouse=True)\n"
        "def _no_real_net_env(monkeypatch):\n"
        "    monkeypatch.setenv('NO_PROXY', '*')\n"
        "    for k in ['HTTP_PROXY','HTTPS_PROXY','http_proxy','https_proxy','ALL_PROXY','all_proxy']:\n"
        "        monkeypatch.delenv(k, raising=False)\n"
        "\n"
        "@pytest.fixture(autouse=True)\n"
        "def _patch_time(monkeypatch):\n"
        "    import time\n"
        "    monkeypatch.setattr(time, 'sleep', lambda *_: None, raising=False)\n"
        "\n"
        "@pytest.fixture(autouse=True)\n"
        "def _patch_temporal(monkeypatch):\n"
        "    try:\n"
        "        import temporalio  # noqa: F401\n"
        "    except Exception:\n"
        "        return\n"
        "    class _Dummy:\n"
        "        def __init__(self, *a, **k): pass\n"
        "        async def __aenter__(self): return self\n"
        "        async def __aexit__(self, *a): pass\n"
        "        def __call__(self, *a, **k): return self\n"
        "        async def run(self, *a, **k): return None\n"
        "        async def start(self, *a, **k): return None\n"
        "    for path in ['temporalio.client.Client','temporalio.worker.Worker','temporalio.workflow.Workflow']:\n"
        "        try:\n"
        "            mod_name, attr = path.rsplit('.', 1)\n"
        "            mod = importlib.import_module(mod_name)\n"
        "            setattr(mod, attr, _Dummy)\n"
        "        except Exception:\n"
        "            pass\n"
        "\n"
        "def _wrap_async(func):\n"
        "    if inspect.iscoroutinefunction(func):\n"
        "        def sync_wrapper(*a, **k):\n"
        "            try:\n"
        "                loop = asyncio.get_event_loop()\n"
        "            except RuntimeError:\n"
        "                loop = asyncio.new_event_loop()\n"
        "                asyncio.set_event_loop(loop)\n"
        "            return loop.run_until_complete(func(*a, **k))\n"
        "        return sync_wrapper\n"
        "    return func\n"
        "\n"
        "_TARGET_MODULES = [\n"
        "    'run_worker_module',\n"
        "    'run_workflow_module',\n"
        "    'run_workflow_main',\n"
        "]\n"
        "\n"
        "@pytest.fixture(autouse=True)\n"
        "def _wrap_async_entrypoints(monkeypatch):\n"
        "    for mod_name in _TARGET_MODULES:\n"
        "        try:\n"
        "            mod = importlib.import_module(mod_name)\n"
        "        except Exception:\n"
        "            continue\n"
        "        for attr in ('main','run','start'):\n"
        "            if hasattr(mod, attr):\n"
        "                try:\n"
        "                    monkeypatch.setattr(mod, attr, _wrap_async(getattr(mod, attr)), raising=False)\n"
        "                except Exception:\n"
        "                    pass\n"
        "    def _safe_run(coro):\n"
        "        if inspect.iscoroutine(coro):\n"
        "            try:\n"
        "                loop = asyncio.get_event_loop()\n"
        "            except RuntimeError:\n"
        "                loop = asyncio.new_event_loop()\n"
        "                asyncio.set_event_loop(loop)\n"
        "            return loop.run_until_complete(coro)\n"
        "        return coro\n"
        "    monkeypatch.setattr(asyncio, 'run', _safe_run, raising=False)\n"
    )
    if cf.exists():
        text = cf.read_text(encoding="utf-8")
        if "_wrap_async_entrypoints" not in text:
            text = text.rstrip() + "\n\n" + payload
            cf.write_text(text, encoding="utf-8")
            print("✅ conftest.py augmented →", cf)
        else:
            print("✅ conftest.py already has autosetup →", cf)
    else:
        cf.write_text(payload, encoding="utf-8")
        print("✅ conftest.py created →", cf)

ensure_pytest_ini()
ensure_conftest_autouse()

# ==== 유틸 ====
def goal_id_from_name(name: str) -> str | None:
    m = RE_GOAL.search(name)
    return m.group("gid") if m else None

def sh(cmd: str, cwd: Path | None = None, timeout: int | None = None, env: dict | None = None):
    try:
        p = subprocess.run(
            cmd, cwd=str(cwd or PROJ), env=env or ENV_BASE,
            shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE,
            timeout=timeout, text=True
        )
        return p.returncode, p.stdout, p.stderr, False
    except subprocess.TimeoutExpired as e:
        return 124, e.stdout or "", e.stderr or "", True

def list_test_files(dir_path: Path) -> list[Path]:
    if not dir_path.exists():
        return []
    files = []
    files += list(dir_path.glob("test*.py"))
    files += list(dir_path.glob("*_test.py"))
    return sorted(set(p for p in files if p.is_file()))

def list_generated_test_files() -> list[Path]:
    if not GEN_DIR.exists():
        return []
    # conftest.py는 수집 대상에서 제외
    return sorted([p for p in GEN_DIR.glob("*.py") if p.is_file() and p.name != "conftest.py"])

def rel_to_proj(p: Path) -> str:
    try:
        return str(p.resolve().relative_to(PROJ))
    except Exception:
        return str(p.resolve())

# ==== 0) 산출물 파일 경로 ====
results_jsonl = ART_DIR / "results.jsonl"
manifest_path = ART_DIR / "manifest.json"
coverage_json_path = ART_DIR / "coverage_gen.json"     # 이번 라운드 통합(프루닝기존+생성)
coverage_xml_path  = ART_DIR / "coverage_gen.xml"
coverage_base_json = ART_DIR / "coverage_base.json"    # 3-1 기준선(원본 기존 테스트)

for old in [results_jsonl, coverage_json_path, coverage_xml_path]:
    if old.exists():
        old.unlink()

runs = []
ok = fail = to_cnt = 0

# ==== 1) (프루닝된) 기존 테스트 개별 격리 실행 ====
base_root = PRUNED_DIR if PRUNED_DIR.exists() else TESTS_DIR
base_files = list_test_files(base_root)

if base_files:
    print(f"🧪 프루닝된 기존 테스트 파일: {len(base_files)}개 @ {rel_to_proj(base_root)}")
    for tf in base_files:
        name = tf.name
        shard = COV_SHARDS_DIR / f".coverage.__base__.{name}"

        env = ENV_BASE.copy()
        env["PYTHONPATH"] = f"{PROJ}:{env.get('PYTHONPATH','')}"
        env["COVERAGE_FILE"] = str(shard)
        env.setdefault("NO_PROXY", "*")

        target = rel_to_proj(tf)
        cmd = f"{PY_EXE} -m coverage run{rc_opt} -m pytest {PYTEST_FLAGS} {shlex.quote(target)}"

        start = time.time()
        ts_start = datetime.now(timezone.utc).isoformat()
        rc, out, err, timed_out = sh(cmd, cwd=PROJ, timeout=TIMEOUT_SEC_BASE, env=env)
        dur = round(time.time() - start, 3)
        ts_end = datetime.now(timezone.utc).isoformat()

        (LOG_DIR / f"__base__{name}.out.txt").write_text(out, encoding="utf-8")
        (LOG_DIR / f"__base__{name}.err.txt").write_text(err, encoding="utf-8")

        runs.append({
            "suite": "BASE",
            "test_file": name,
            "goal_id": None,
            "start_utc": ts_start,
            "end_utc": ts_end,
            "duration_sec": dur,
            "returncode": rc,
            "timed_out": timed_out,
            "stdout_len": len(out),
            "stderr_len": len(err),
            "shard_path": str(shard),
            "invoked_path": target,
        })

        if timed_out:
            to_cnt += 1
            print(f"⏱️ TIMEOUT [BASE] {name} ({dur}s)")
        elif rc == 0:
            ok += 1
            print(f"✅ PASS   [BASE] {name} ({dur}s)")
        else:
            fail += 1
            first_err = (err.strip().splitlines() or [''])[0]
            print(f"❌ FAIL   [BASE] {name} (rc={rc}, {dur}s) :: {first_err}")
else:
    print("ℹ️ 프루닝된 기존 테스트가 없어 BASE 실행을 건너뜁니다. (Pruned_Base_Tests/ 또는 tests/ 비어있음)")

# ==== 2) 생성 테스트 파일 개별 격리 실행 ====
test_files = list_generated_test_files()
print(f"🧪 생성 테스트 파일: {len(test_files)}개 @ {rel_to_proj(GEN_DIR)}")
for tf in test_files:
    name = tf.name
    gid = goal_id_from_name(name) or "----"
    shard = COV_SHARDS_DIR / f".coverage.{name}"

    env = ENV_BASE.copy()
    env["PYTHONPATH"] = f"{PROJ}:{env.get('PYTHONPATH','')}"
    env["COVERAGE_FILE"] = str(shard)
    env.setdefault("NO_PROXY", "*")

    target = rel_to_proj(tf)   # e.g. "generated_tests/test_0001_...py"
    cmd = f"{PY_EXE} -m coverage run{rc_opt} -m pytest {PYTEST_FLAGS} {shlex.quote(target)}"

    start = time.time()
    ts_start = datetime.now(timezone.utc).isoformat()
    rc, out, err, timed_out = sh(cmd, cwd=PROJ, timeout=TIMEOUT_SEC_GEN, env=env)
    dur = round(time.time() - start, 3)
    ts_end = datetime.now(timezone.utc).isoformat()

    (LOG_DIR / f"{name}.out.txt").write_text(out, encoding="utf-8")
    (LOG_DIR / f"{name}.err.txt").write_text(err, encoding="utf-8")

    runs.append({
        "suite": "GEN",
        "test_file": name,
        "goal_id": gid,
        "start_utc": ts_start,
        "end_utc": ts_end,
        "duration_sec": dur,
        "returncode": rc,
        "timed_out": timed_out,
        "stdout_len": len(out),
        "stderr_len": len(err),
        "shard_path": str(shard),
        "invoked_path": target,
    })

    if timed_out:
        to_cnt += 1
        print(f"⏱️ TIMEOUT [GEN] {name} ({dur}s)")
    elif rc == 0:
        ok += 1
        print(f"✅ PASS   [GEN] {name} ({dur}s)")
    else:
        fail += 1
        first_err = (err.strip().splitlines() or [''])[0]
        print(f"❌ FAIL   [GEN] {name} (rc={rc}, {dur}s) :: {first_err}")

# 실행 기록 저장
results_jsonl.write_text(
    "\n".join(json.dumps(r, ensure_ascii=False) for r in runs) + ("\n" if runs else ""),
    encoding="utf-8"
)

manifest = {
    "generated_at_utc": datetime.now(timezone.utc).isoformat(),
    "project": str(PROJ),
    "run_dir": str(ART_DIR),
    "tests_total": len([r for r in runs if r['suite'] in {'BASE','GEN'}]),
    "pass": ok,
    "fail": fail,
    "timeout": to_cnt,
    "logs_dir": str(LOG_DIR),
    "cov_shards_dir": str(COV_SHARDS_DIR),
    "base_root": str(rel_to_proj(base_root)) if base_files else None,
    "gen_root": str(rel_to_proj(GEN_DIR)),
}
manifest_path.write_text(json.dumps(manifest, ensure_ascii=False, indent=2), encoding="utf-8")
print("\n📦 실행 요약:", json.dumps({"pass": ok, "fail": fail, "timeout": to_cnt, "total": manifest['tests_total']}, ensure_ascii=False))

# ==== 3) 커버리지 결합(JSON/XML/HTML) ====
shards = sorted([p for p in COV_SHARDS_DIR.iterdir() if p.name.startswith(".coverage.")])
if not shards:
    print("⚠️ 커버리지 샤드가 없습니다. 테스트가 즉시 실패했을 수 있습니다.")
else:
    subprocess.call(f"coverage erase{rc_opt}", shell=True, cwd=str(PROJ))
    combine_cmd = "coverage combine" + rc_opt + " " + " ".join(shlex.quote(str(p)) for p in shards)
    print("> ", combine_cmd)
    subprocess.call(combine_cmd, shell=True, cwd=str(PROJ))
    subprocess.call(f"coverage json -o {coverage_json_path.name}{rc_opt}", shell=True, cwd=str(PROJ))
    subprocess.call(f"coverage xml  -o {coverage_xml_path.name}{rc_opt}",  shell=True, cwd=str(PROJ))
    subprocess.call(f"coverage html -d {HTML_DIR_GEN.name}{rc_opt}",       shell=True, cwd=str(PROJ))

    # 결과 파일을 run_artifacts에 복사 보관
    src_json = PROJ / coverage_json_path.name
    src_xml  = PROJ / coverage_xml_path.name
    if src_json.exists(): shutil.copy2(src_json, coverage_json_path)
    if src_xml.exists():  shutil.copy2(src_xml,  coverage_xml_path)

    print("✅ 커버리지 결합 완료")
    print(" - JSON :", coverage_json_path)
    print(" - XML  :", coverage_xml_path)
    print(" - HTML :", HTML_DIR_GEN / "index.html")

# ==== 4) 분기 관측/목표 달성률 계산 ====
observed_outcomes_gen = {}
branch_points = full_hit = half_hit = zero_hit = 0

if coverage_xml_path.exists():
    try:
        xml_root = etree.parse(str(coverage_xml_path)).getroot()
        for cls in xml_root.findall(".//class"):
            filename = cls.get("filename") or ""
            if not filename:
                continue
            abs_path = (PROJ / filename).resolve() if not Path(filename).is_absolute() else Path(filename)
            for line in cls.findall("./lines/line"):
                if line.get("branch") != "true":
                    continue
                try:
                    num = int(line.get("number"))
                except Exception:
                    continue
                cond = line.get("condition-coverage")  # "50% (1/2)"
                covered = total = 0
                if cond:
                    m = re.search(r"\((\d+)\s*/\s*(\d+)\)", cond)
                    if m:
                        covered, total = int(m.group(1)), int(m.group(2))
                if total == 0:
                    continue
                observed_outcomes_gen.setdefault(str(abs_path), {})[num] = {
                    "covered": covered, "total": total, "ratio": round(covered/total, 3)
                }
                branch_points += 1
                if covered == 0: zero_hit += 1
                elif covered == total: full_hit += 1
                else: half_hit += 1
    except Exception as e:
        print("⚠️ coverage_xml 파싱 실패:", e)

(ART_DIR / "observed_outcomes_gen.json").write_text(
    json.dumps(observed_outcomes_gen, ensure_ascii=False, indent=2),
    encoding="utf-8"
)

print(f"🧮 분기 관측 요약 → total:{branch_points}, full:{full_hit}, half:{half_hit}, zero:{zero_hit}")

# ==== 5) 목표 달성률(기본) ====
GOALS_FILE = ART_DIR / "goals_ranked.json"
if GOALS_FILE.exists() and coverage_json_path.exists():
    cov_json = json.loads((coverage_json_path).read_text(encoding="utf-8"))
    files_map = cov_json.get("files", {}) or {}

    def line_hit(fpath: str, ln: int) -> bool:
        finfo = files_map.get(fpath) or files_map.get(str(Path(fpath).resolve()))
        if not finfo:
            return False
        executed = set(finfo.get("executed_lines", []) or [])
        return ln in executed

    goals = json.loads(GOALS_FILE.read_text(encoding="utf-8"))
    goal_stats = []
    for g in goals:
        f = g["file"]
        abs1 = str((PROJ / f).resolve())
        abs2 = f
        hit = sum(1 for ln in g.get("target_lines", []) if line_hit(abs1, ln) or line_hit(abs2, ln))
        total = len(g.get("target_lines", [])) or 1
        goal_stats.append({"id": g["id"], "hit": hit, "total": total, "rate": round(hit/total, 3)})

    (ART_DIR / "goal_achievements.json").write_text(
        json.dumps(goal_stats, ensure_ascii=False, indent=2),
        encoding="utf-8"
    )
    hit_goals = sum(1 for s in goal_stats if s["hit"] > 0)
    print(f"🎯 목표 달성률: {hit_goals}/{len(goal_stats)} 목표가 ≥1 라인 도달")

# ==== 6) 베이스라인 대비 향상치(delta) 계산 ====
def load_json(p: Path, default=None):
    try:
        return json.loads(p.read_text(encoding="utf-8"))
    except Exception:
        return default

base = load_json(coverage_base_json, {"files": {}}) or {"files": {}}
gen  = load_json(coverage_json_path, {"files": {}}) or {"files": {}}
base_files = base.get("files", {}) or {}
gen_files  = gen.get("files", {})  or {}

def _sum_len(key, d):
    return sum(len((d.get(f, {}) or {}).get(key, []) or []) for f in d.keys())

base_exec = _sum_len("executed_lines", base_files)
base_miss = _sum_len("missing_lines",  base_files)
gen_exec  = _sum_len("executed_lines", gen_files)
gen_miss  = _sum_len("missing_lines",  gen_files)

delta = {
    "executed_lines_delta": gen_exec - base_exec,
    "missing_lines_delta":  base_miss - gen_miss,   # +면 미싱 감소
    "base_executed": base_exec,
    "gen_executed":  gen_exec,
    "base_missing":  base_miss,
    "gen_missing":   gen_miss,
}
(ART_DIR / "coverage_delta.json").write_text(json.dumps(delta, ensure_ascii=False, indent=2), encoding="utf-8")
print("📈 베이스라인 대비 향상치:", json.dumps(delta, ensure_ascii=False))

print("✅ 3-5 완료: (프루닝기존+생성) autosetup + 격리 실행/샤드 결합/분기·목표·향상치 산출")


✅ pytest.ini ensured → /content/money-transfer-project-template-python/pytest.ini
✅ conftest.py created → /content/money-transfer-project-template-python/generated_tests/conftest.py
🧪 프루닝된 기존 테스트 파일: 1개 @ Pruned_Base_Tests
✅ PASS   [BASE] test_run_worker.py (15.539s)
🧪 생성 테스트 파일: 8개 @ generated_tests
✅ PASS   [GEN] test_gen_0001_activities_refund_1.py (14.203s)
✅ PASS   [GEN] test_gen_0001_activities_refund_2.py (14.145s)
✅ PASS   [GEN] test_gen_0002_activities_refund_1.py (14.451s)
✅ PASS   [GEN] test_gen_0002_activities_refund_2.py (14.444s)
✅ PASS   [GEN] test_gen_0003_activities_refund_1.py (14.48s)
✅ PASS   [GEN] test_gen_0003_activities_refund_2.py (15.281s)
❌ FAIL   [GEN] test_gen_0010_activities_deposit_1.py (rc=1, 14.561s) :: 
❌ FAIL   [GEN] test_gen_0010_activities_deposit_2.py (rc=1, 14.568s) :: 

📦 실행 요약: {"pass": 7, "fail": 2, "timeout": 0, "total": 9}
>  coverage combine --rcfile /content/money-transfer-project-template-python/.coveragerc /content/money-transfer-project-t

In [12]:
#@title 3-6) 테스트 보완 – Improvement-based Adaptive Round (coverage_gen 기반 적응형 선별·분포 주입)
import os, sys, json, re
from pathlib import Path

# ==== 경로/상수 ====
assert 'PROJ' in globals(), "3-0 단계를 먼저 실행하세요."
PROJ = Path(PROJ).resolve()
ART_DIR = PROJ / "run_artifacts" / "run1"
GEN_DIR = PROJ / "generated_tests"
LOG_DIR = ART_DIR / "logs"
HTML_GEN_DIR = PROJ / "htmlcov_gen"

COV_BASE_JSON = ART_DIR / "coverage_base.json"       # 3-1 기준선 (참고용)
COV_GEN_JSON  = ART_DIR / "coverage_gen.json"        # 3-5/3-8 통합 커버리지(필수)
GOALS_FILE    = ART_DIR / "goals_ranked.json"        # 3-2 목표
UNCV_MAP_JSON = ART_DIR / "uncovered_map_base.json"  # 3-1 미커버 라인(참고용)
RESULTS_JL    = ART_DIR / "results.jsonl"            # 3-5/3-8 실행 결과 로그 인덱스(참고용)

# 선별/배치 파라미터
NEAR_MISS_WINDOW = 2
BATCH_SIZE = 3
MAX_ROUNDS = 5
TOPK_FILES = 10   # 전역 미커버 상위 파일 요약 개수

# ==== 적응형 임계/수렴 파라미터(환경변수로 조정 가능) ====
TAU1_RATE = float(os.environ.get("ADAPT_TAU1_RATE", "0.25"))  # 목표달성률 증가 임계(예: 0.25=25%p)
CONV_EPS  = float(os.environ.get("ADAPT_CONV_EPS", "0.01"))   # (참고) 전체 라인커버리지 수렴 판단 1%p
CONV_PATIENCE = int(os.environ.get("ADAPT_CONV_PATIENCE", "1"))

# ==== 라운드 디렉터리 자동 증가 ====
def next_round_dir(base: Path) -> Path:
    i = 1
    while True:
        cand = base / f"refine_round{i}"
        if not cand.exists():
            cand.mkdir(parents=True, exist_ok=True)
            return cand
        i += 1

REFINE_DIR = next_round_dir(ART_DIR)
REFINE_PROMPTS = REFINE_DIR / "llm_refine_prompts.jsonl"
REFINE_SUMMARY = REFINE_DIR / "refine_selection.json"
REFINE_TEST_EXPORT = REFINE_DIR / "selected_tests_dump.json"

# ==== 유틸 ====
def load_json(p: Path, default=None):
    try:
        return json.loads(p.read_text(encoding="utf-8"))
    except Exception:
        return default

def read_text_safe(p: Path) -> str:
    try:
        return p.read_text(encoding="utf-8")
    except Exception:
        return ""

def first_lines(s: str, n=2000):
    if not s:
        return ""
    head = s[:n]
    if len(s) > n:
        head += "\n...<truncated>..."
    return head

RE_GID = re.compile(r"(?:^|[_-])(?P<gid>\d{4})(?:[_-]|$)")
def goal_id_from_name(name: str) -> str | None:
    m = RE_GID.search(name)
    return m.group("gid") if m else None

def list_generated_tests_for_gid(gid: str) -> list[Path]:
    """goal id에 해당하는 생성/보강 테스트 후보들을 최신순으로 반환."""
    if not GEN_DIR.exists():
        return []
    cands = [p for p in GEN_DIR.glob("*.py") if gid in p.name]
    # 보강본(*_rN.py)을 우선, 숫자 큰 것 우선 → 없으면 기본 파일
    def rank(p: Path):
        m = re.search(r"_r(\d+)\.py$", p.name)
        r = int(m.group(1)) if m else -1
        return (0 if r >= 0 else 1, -r, -p.stat().st_mtime)
    return sorted(cands, key=rank)

def pick_latest_test_for_gid(gid: str) -> Path | None:
    lst = list_generated_tests_for_gid(gid)
    return lst[0] if lst else None

# ==== 이전 라운드 스냅샷/히스토리 로더 ====
def _load_latest_two_histories():
    hist_path = ART_DIR / "history.jsonl"
    if not hist_path.exists():
        return None, None
    rows = [json.loads(x) for x in hist_path.read_text(encoding="utf-8").splitlines() if x.strip()]
    if len(rows) == 0: return None, None
    if len(rows) == 1: return rows[-1], None
    return rows[-1], rows[-2]

def _goal_rate_map(hist_row):
    """history.jsonl의 goal_achievements를 id->rate 맵으로 변환"""
    if not hist_row or not hist_row.get("goal_achievements"):
        return {}
    m = {}
    for g in hist_row["goal_achievements"]:
        m[str(g["id"])] = float(g.get("rate", 0.0))
    return m

# ==== 실패유형 분류기(로그 헤드 기반) ====
_ERR_PATTERNS = {
    "import_error":  [r"ModuleNotFoundError", r"ImportError"],
    "timeout":       [r"TimeoutExpired", r"timed out"],
    "crash":         [r"Segmentation fault", r"Fatal Python error"],
    "assert_fail":   [r"AssertionError", r"assert .* failed"],
    "runtime_warn":  [r"RuntimeWarning", r"coroutine .* was never awaited"],
}
def classify_failure(stdout_head: str, stderr_head: str) -> str|None:
    blob = (stderr_head or "") + "\n" + (stdout_head or "")
    for tag, pats in _ERR_PATTERNS.items():
        for p in pats:
            if re.search(p, blob):
                return tag
    return None

# ==== 데이터 로드 ====
cov_base = load_json(COV_BASE_JSON, {"files": {}}) or {"files": {}}
cov_gen  = load_json(COV_GEN_JSON,  {"files": {}}) or {"files": {}}
goals    = load_json(GOALS_FILE,    []) or []
uncovered_map = load_json(UNCV_MAP_JSON, {}) or {}

if not cov_gen or not goals:
    raise SystemExit("필수 산출물(coverage_gen.json 또는 goals_ranked.json)이 없습니다. 3-2, 3-5/3-8 후 실행하세요.")

gen_files = cov_gen.get("files", {}) or {}

def _info_for(file_rel: str):
    """coverage_gen.json에서 상대/절대 키 모두 탐색."""
    return gen_files.get(file_rel) or gen_files.get(str((PROJ / file_rel).resolve()))

def line_hit_in_gen(file_rel: str, ln: int) -> bool:
    info = _info_for(file_rel)
    if not info:
        return False
    return ln in set(info.get("executed_lines", []) or [])

def executed_set_in_gen(file_rel: str) -> set[int]:
    info = _info_for(file_rel)
    return set(info.get("executed_lines", []) or []) if info else set()

def missing_set_in_gen(file_rel: str) -> set[int]:
    info = _info_for(file_rel)
    return set(info.get("missing_lines", []) or []) if info else set()

# ==== A) coverage_gen.json에서 전역 미커버 라인 분포 추출 ====
global_uncovered_map = {}
total_missing = 0
for fkey, finfo in gen_files.items():
    miss = sorted(set((finfo or {}).get("missing_lines", []) or []))
    if miss:
        global_uncovered_map[fkey] = miss
        total_missing += len(miss)

global_uncovered_summary = []
for fkey, miss in sorted(global_uncovered_map.items(), key=lambda kv: len(kv[1]), reverse=True)[:TOPK_FILES]:
    global_uncovered_summary.append({
        "file": fkey,
        "missing_count": len(miss),
        "missing_lines": miss[:200],
    })

# ==== 1) 미도달 목표 판정 (coverage_gen 기준) ====
miss_goals = []  # 1차 후보
for g in goals:
    file_rel = g["file"]
    tlines = g.get("target_lines", []) or []
    hits = sum(1 for ln in tlines if line_hit_in_gen(file_rel, ln))
    if hits == 0 and tlines:  # 목표 라인 ≥1줄도 못 맞춘 경우만 보강
        exed = executed_set_in_gen(file_rel)
        neigh = set()
        for t in tlines:
            for k in range(-NEAR_MISS_WINDOW, NEAR_MISS_WINDOW + 1):
                neigh.add(t + k)
        near = len(exed & neigh) > 0
        miss_goals.append({
            "goal_id": g["id"],
            "file": file_rel,
            "function": g.get("function", {}),
            "target_lines": tlines,
            "near_miss": near
        })

# ==== 1.5) 적응형 선별: 이전 라운드 대비 목표달성률 Δ와 실패유형 평가 ====
latest, prev = _load_latest_two_histories()
rate_now = _goal_rate_map(latest)
rate_prev = _goal_rate_map(prev)

# results 인덱스 로드(마지막 라운드 실행 결과)
results_idx = []
if RESULTS_JL.exists():
    results_idx = [json.loads(x) for x in RESULTS_JL.read_text(encoding="utf-8").splitlines() if x.strip()]

# test_file -> (stdout, stderr) 헤드 미리 읽기
def _load_heads(tname: str):
    return (
        read_text_safe(LOG_DIR / f"{tname}.out.txt"),
        read_text_safe(LOG_DIR / f"{tname}.err.txt"),
    )

# goal_id → 최신 테스트 파일명 추정
def _guess_last_test_name_for_gid(gid: str) -> str|None:
    p = pick_latest_test_for_gid(gid)
    return p.name if p else None

adaptive_candidates = []
for item in miss_goals:
    gid = str(item["goal_id"])
    now = float(rate_now.get(gid, 0.0))
    prv = float(rate_prev.get(gid, 0.0))
    delta_rate = now - prv

    # 마지막 실행의 실패 유형 확인(있으면 제외 규칙 적용)
    last_tname = _guess_last_test_name_for_gid(gid)
    fclass = None
    if last_tname:
        out_h, err_h = _load_heads(last_tname)
        fclass = classify_failure(out_h, err_h)

    fatal = fclass in {"import_error", "timeout", "crash"}  # 제외 기준

    if (delta_rate >= TAU1_RATE) or item["near_miss"]:
        if not fatal:
            item["delta_rate"] = round(delta_rate, 3)
            item["last_failure"] = fclass
            adaptive_candidates.append(item)

# 이후 파이프라인은 adaptive_candidates 사용
miss_goals = adaptive_candidates

# ==== 2) 각 미도달 goal → 최신 테스트 파일 매핑 & 프롬프트 입력 구성 ====
selected = []
for item in miss_goals:
    gid = item["goal_id"]
    tfile = pick_latest_test_for_gid(gid)
    if not tfile:
        # 생성된 테스트가 없으면 스킵(다음 라운드에 새로 생성)
        continue

    original_code = read_text_safe(tfile)
    out_log = read_text_safe(LOG_DIR / f"{tfile.name}.out.txt")
    err_log = read_text_safe(LOG_DIR / f"{tfile.name}.err.txt")

    # 해당 목표에서 아직 미도달한 타겟 라인(coverage_gen 기준)
    still_missing = [ln for ln in item["target_lines"] if not line_hit_in_gen(item["file"], ln)]

    # 현재 라운드 기준 그 파일의 전체 미커버 라인(coverage_gen 기준)
    file_uncovered_remaining_gen = sorted(missing_set_in_gen(item["file"]))

    # baseline의 파일별 미커버 라인(참고용)
    file_abs = str((PROJ / item["file"]).resolve())
    base_uncovered = uncovered_map.get(file_abs, uncovered_map.get(item["file"], [])) or []

    selected.append({
        "id": f"{gid}::{tfile.name}",
        "goal_id": gid,
        "target_file": item["file"],
        "target_lines": item["target_lines"],
        "near_miss": item["near_miss"],
        "delta_rate": item.get("delta_rate"),
        "last_failure": item.get("last_failure"),
        "uncovered_diff": {
            "still_missing_target_lines": sorted(still_missing),
            "file_uncovered_lines_baseline": sorted(set(int(x) for x in base_uncovered)),
            "file_uncovered_remaining_gen": file_uncovered_remaining_gen,   # ★ 현재 미커버(해당 파일)
        },
        "run": {
            "stdout_head": first_lines(out_log, 1500),
            "stderr_head": first_lines(err_log, 1500),
        },
        "original_test_code": original_code,
    })

# 선별 없으면 종료
if not selected:
    REFINE_PROMPTS.write_text("", encoding="utf-8")
    REFINE_SUMMARY.write_text(json.dumps({
        "round_dir": REFINE_DIR.name,
        "using_coverage_gen_only": True,
        "selected": 0,
        "reason": "적응형 기준(Δ-rate≥τ1 또는 near-miss) 충족 목표 없음, 혹은 최신 테스트 파일 부재",
        "params": {
            "tau1_rate": TAU1_RATE,
            "near_miss_window": NEAR_MISS_WINDOW
        },
        "global_uncovered": {
            "total_missing_lines": total_missing,
            "top_files": global_uncovered_summary
        }
    }, ensure_ascii=False, indent=2), encoding="utf-8")
    print("ℹ️ 보강 대상이 없습니다. 프롬프트를 생성하지 않았습니다.")
    raise SystemExit(0)

# ==== 3) 파일별 충돌 피하기(동일 파일 목표는 한 배치에 하나만) → 배치 구성 ====
batches, bucket, seen_files = [], [], set()
for rec in selected:
    f = rec["target_file"]
    if f in seen_files or len(bucket) >= BATCH_SIZE:
        if bucket:
            batches.append(bucket)
        bucket, seen_files = [], set()
    bucket.append(rec)
    seen_files.add(f)
if bucket:
    batches.append(bucket)

# ==== 4) LLM 보강 프롬프트(JSONL) 생성 ====
SYSTEM_REFINE = (
    "당신은 기존 pytest 테스트를 보강하여 미커버 영역(Target Lines)에 도달하도록 수정하는 전문가입니다.\n"
    "출력은 마크다운 없이 **순수 JSON 객체** 하나로만 응답합니다. 스키마는 아래와 같습니다:\n"
    "{\n"
    '  "edits": [\n'
    '    {"id": "<goal_id::filename>", "new_code": "<보강된 pytest 테스트 파일 전체 문자열>"}\n'
    "  ]\n"
    "}\n"
    "지침:\n"
    "• 테스트의 구조를 유지하되, 입력/경계조건/호출 순서/예외 트리거를 조정해 `still_missing_target_lines`에 실제로 도달하게 하세요.\n"
    "• 외부 부작용 금지(파일/네트워크/시간/환경/Temporal). 필요한 경우 monkeypatch/더블을 사용하세요.\n"
    "• import는 importlib + getattr 경로를 유지하고, 속성이 없을 때만 가드형 조건으로 pytest.skip을 허용합니다.\n"
    "• 각 테스트는 최소 1줄 이상의 target_lines를 실제 실행해야 하며, 관련된 assert 또는 pytest.raises를 포함해야 합니다.\n"
    "• 테스트 이름에 타격 라인을 `hits_L<line>` 형태로 포함하는 것을 권장합니다.\n"
    "• 출력에는 코드 외 설명/주석/마크다운을 포함하지 마세요. **JSON만** 반환하세요.\n"
)

with REFINE_PROMPTS.open("w", encoding="utf-8") as outf:
    for i, batch in enumerate(batches, start=1):
        user_payload = {
            "schema_version": "refine-v1",
            "round_dir": REFINE_DIR.name,
            "selection_params": {
                "tau1_rate": TAU1_RATE,
                "near_miss_window": NEAR_MISS_WINDOW,
                "batch_size": BATCH_SIZE,
                "using_coverage_gen_only": True,
                "max_rounds": MAX_ROUNDS
            },
            # ★ 전역 미커버 요약(모델이 우선순위 고려하도록 힌트)
            "global_uncovered": {
                "total_missing_lines": total_missing,
                "top_files": global_uncovered_summary
            },
            "batch_index": i,
            "tests": batch,
        }
        record = {
            "meta": {
                "batch_index": i,
                "num_tests": len(batch),
                "ids": [t["id"] for t in batch],
                "round_dir": REFINE_DIR.name,
                "using_coverage_gen_only": True,
            },
            "messages": [
                {"role": "system", "content": SYSTEM_REFINE},
                {"role": "user", "content": json.dumps(user_payload, ensure_ascii=False, indent=2)},
            ],
        }
        outf.write(json.dumps(record, ensure_ascii=False) + "\n")

# ==== 5) 요약/덤프 ====
summary = {
    "round_dir": REFINE_DIR.name,
    "using_coverage_gen_only": True,
    "params": {
        "tau1_rate": TAU1_RATE,
        "near_miss_window": NEAR_MISS_WINDOW,
        "batch_size": BATCH_SIZE
    },
    "selected": len(selected),
    "batches": [{"batch_index": i+1, "num_tests": len(b)} for i, b in enumerate(batches)],
    "global_uncovered": {
        "total_missing_lines": total_missing,
        "top_files": global_uncovered_summary
    }
}
REFINE_SUMMARY.write_text(json.dumps(summary, ensure_ascii=False, indent=2), encoding="utf-8")
REFINE_TEST_EXPORT.write_text(json.dumps(selected, ensure_ascii=False, indent=2), encoding="utf-8")

print("✅ 보강 대상 선별 완료 (Δ-rate≥τ1/near-miss + fatal 실패 제외 + 전역 분포 주입)")
print(" - 라운드 폴더:", REFINE_DIR)
print(" - 선별된 테스트 수:", len(selected))
print(" - 배치 수:", len(batches))
print(" - 프롬프트 JSONL:", REFINE_PROMPTS)
print(" - 선별 요약:", REFINE_SUMMARY)


✅ 보강 대상 선별 완료 (Δ-rate≥τ1/near-miss + fatal 실패 제외 + 전역 분포 주입)
 - 라운드 폴더: /content/money-transfer-project-template-python/run_artifacts/run1/refine_round2
 - 선별된 테스트 수: 3
 - 배치 수: 3
 - 프롬프트 JSONL: /content/money-transfer-project-template-python/run_artifacts/run1/refine_round2/llm_refine_prompts.jsonl
 - 선별 요약: /content/money-transfer-project-template-python/run_artifacts/run1/refine_round2/refine_selection.json


In [13]:
#@title 3-7) 보강 적용기 – 경로 버그 수정 + GOAL_LUT 호환 + 디버그 로그 강화
import os, re, json, ast, shutil, time, subprocess, shlex, sys
from pathlib import Path
from datetime import datetime, timezone
import httpx
import backoff
from lxml import etree
from openai import OpenAI, APIError, RateLimitError, APIConnectionError

# ==== 설정 ====
REPLACE_IN_PLACE = os.getenv("REFINE_REPLACE_IN_PLACE", "true").lower() in {"1","true","yes"}
MODEL = os.getenv("REFINE_MODEL", "gpt-4o")
TIMEOUT_SINGLE = int(os.getenv("REFINE_SINGLE_TIMEOUT", "30"))
RCFILE = Path(os.getenv("REFINE_RCFILE", ".coveragerc"))
MIN_EXEC_GAIN = int(os.getenv("REFINE_MIN_EXEC_GAIN", "3"))  # τ₂
PYTEST_FLAGS = os.getenv("REFINE_PYTEST_FLAGS", "-q -s")

# ==== 경로 ====
assert 'PROJ' in globals(), "3-0 단계를 먼저 실행하세요."
PROJ = Path(PROJ).resolve()
ART_DIR = PROJ / "run_artifacts" / "run1"
GEN_DIR = PROJ / "generated_tests"
REFINE_DIRS = sorted([p for p in ART_DIR.iterdir() if p.is_dir() and p.name.startswith("refine_round")])
if not REFINE_DIRS:
    raise SystemExit("refine_roundN 폴더가 없습니다. 3-6을 먼저 실행하세요.")
REFINE_DIR = REFINE_DIRS[-1]
PROMPTS_PATH = REFINE_DIR / "llm_refine_prompts.jsonl"
if not PROMPTS_PATH.exists():
    raise SystemExit(f"프롬프트 파일이 없습니다: {PROMPTS_PATH}")
if PROMPTS_PATH.stat().st_size == 0:
    print("ℹ️ 보강 프롬프트가 비어있습니다. 적용 단계 건너뜁니다."); raise SystemExit(0)

RAW_DIR = REFINE_DIR / "_raw_edits"
ERR_DIR = REFINE_DIR / "_errors"
ARCHIVE_DIR = REFINE_DIR / "_archive"
REJECT_DIR = REFINE_DIR / "_rejected"
STAGE_DIR  = REFINE_DIR / "_staging"
LOG_PATH   = REFINE_DIR / "apply_log.jsonl"
MANIFEST   = GEN_DIR / "ACTIVE_MANIFEST.json"

for d in (RAW_DIR, ERR_DIR, ARCHIVE_DIR, REJECT_DIR, STAGE_DIR, GEN_DIR):
    d.mkdir(parents=True, exist_ok=True)

# ==== OpenAI ====
if not os.getenv("OPENAI_API_KEY"):
    raise RuntimeError("OPENAI_API_KEY가 설정되지 않았습니다. 3-0 단계에서 .env를 로드했는지 확인하세요.")
http_client = httpx.Client(timeout=180.0, follow_redirects=True,
                           limits=httpx.Limits(max_connections=1, max_keepalive_connections=0),
                           transport=httpx.HTTPTransport(retries=5))
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"),
                base_url="https://api.openai.com/v1", http_client=http_client)

# ==== 유틸 ====
def now_ts(): return time.strftime("%Y%m%d_%H%M%S")
def strip_fences(s: str) -> str:
    s = re.sub(r"^```[a-zA-Z0-9]*\s*", "", (s or "").strip()); s = re.sub(r"\s*```$", "", s); return s
def write_error(tag: str, payload: dict):
    (ERR_DIR / f"{tag}.json").write_text(json.dumps(payload, ensure_ascii=False, indent=2), encoding="utf-8")
def ensure_unique_path(base: Path) -> Path:
    p = base; i = 2
    while p.exists(): p = base.with_name(f"{base.stem}_{i}{base.suffix}"); i += 1
    return p

# 러너/메인가드 제거
def sanitize_test_code(code: str) -> str:
    pats = [
        re.compile(r"(?ms)^\s*if\s+__name__\s*==\s*['\"]__main__['\"]\s*:\s*\n(?:\s*.*\n?)+$"),
        re.compile(r"(?m)^\s*pytest\.main\s*\(.*?\)\s*$"),
        re.compile(r"(?m)^\s*unittest\.main\s*\(.*?\)\s*$"),
    ]
    new = code or ""
    for p in pats: new = p.sub("", new)
    return (new.strip() + "\n") if new.strip() else "\n"

# ---- 검증 (가드형 skip만 허용) ----
RE_IMPORTLIB = re.compile(r"\bimportlib\.import_module\s*\(")
RE_PYTEST_RAISES = re.compile(r"\bpytest\.raises\s*\(")
RE_GETATTR = re.compile(r"\bgetattr\s*\(\s*[^,]+,\s*['\"][A-Za-z_][A-Za-z0-9_]*['\"]")

def parse_ast_or_error(code: str):
    try: return ast.parse(code), None
    except SyntaxError as e: return None, f"syntax_error:{e.msg}@L{e.lineno}"

def extract_test_funcs(tree: ast.AST):
    return [n for n in ast.walk(tree) if isinstance(n, ast.FunctionDef) and n.name.startswith("test_")]

def has_assert_or_raises(tree: ast.AST, code: str) -> bool:
    return any(isinstance(n, ast.Assert) for n in ast.walk(tree)) or bool(RE_PYTEST_RAISES.search(code))

def uses_importlib(code: str) -> bool: return bool(RE_IMPORTLIB.search(code))
def uses_getattr_guardish(code: str) -> bool: return bool(RE_GETATTR.search(code or ""))

def _parent_map(tree: ast.AST):
    parent = {}
    for node in ast.walk(tree):
        for child in ast.iter_child_nodes(node): parent[child] = node
    return parent

def _is_guarded_skip(call: ast.Call, parent_map) -> bool:
    cur = call
    while cur in parent_map:
        cur = parent_map[cur]
        if isinstance(cur, ast.If):
            t = cur.test
            if (isinstance(t, ast.Compare) and len(t.ops)==1 and isinstance(t.ops[0], ast.Is)
                and len(t.comparators)==1 and isinstance(t.comparators[0], ast.Constant)
                and t.comparators[0].value is None): return True
        if isinstance(cur, ast.ExceptHandler):
            t = cur.type
            if isinstance(t, ast.Name) and t.id in {"ImportError","NameError"}: return True
            if isinstance(t, ast.Tuple) and any(isinstance(e, ast.Name) and e.id in {"ImportError","NameError"} for e in t.elts): return True
    return False

def has_unconditional_skip(code: str):
    try: tree = ast.parse(code)
    except SyntaxError: return (False, [])
    parent = _parent_map(tree); bad = []
    for node in ast.walk(tree):
        if isinstance(node, ast.Call):
            f = node.func
            if isinstance(f, ast.Attribute) and isinstance(f.value, ast.Name) and f.value.id=="pytest" and f.attr=="skip":
                if not _is_guarded_skip(node, parent): bad.append(getattr(node, "lineno", -1))
    return (len(bad)>0), bad

def minimal_viability_checks(code: str):
    reasons = []; meta = {"warnings": []}
    if len((code or "").strip()) < 60: reasons.append("too_short")
    tree, synerr = parse_ast_or_error(code)
    if synerr: reasons.append(synerr); return False, reasons, meta
    tests = extract_test_funcs(tree)
    if not tests: reasons.append("no_test_functions")
    if not has_assert_or_raises(tree, code): reasons.append("no_assert_or_raises")
    if not uses_importlib(code): reasons.append("no_importlib_import_module")
    if tests and not any("hits_L" in t.name for t in tests): meta["warnings"].append("missing_hits_L_in_test_name")
    bad_skip, lines = has_unconditional_skip(code)
    if bad_skip: reasons.append(f"unconditional_skip_detected@{lines}")
    if not uses_getattr_guardish(code): meta["warnings"].append("missing_getattr_guardish_hint")
    return (len(reasons) == 0), reasons, meta

# ==== OpenAI 호출 ====
@backoff.on_exception(backoff.expo, (APIConnectionError, APIError, RateLimitError), max_tries=8, max_time=300)
def call_openai_with_retry(messages):
    resp = client.chat.completions.create(
        model=MODEL, messages=messages,
        response_format={"type": "json_object"}, timeout=180.0,
    )
    return resp.choices[0].message.content

# ==== 활성 매니페스트 ====
def load_manifest():
    try: return json.loads(MANIFEST.read_text(encoding="utf-8"))
    except Exception: return {"active": {}, "history": {}}
def save_manifest(m): MANIFEST.write_text(json.dumps(m, ensure_ascii=False, indent=2), encoding="utf-8")
manifest = load_manifest()

# ==== 프롬프트 → goal 메타 복원 (호환형 파서) ====
def load_goal_lookup():
    gid2spec = {}
    with PROMPTS_PATH.open("r", encoding="utf-8") as f:
        for line in f:
            rec = json.loads(line)
            user_content = rec["messages"][1]["content"]
            try:
                payload = json.loads(user_content)
            except Exception:
                continue
            # 1) 최신 스키마: edits 대상 테스트 배열
            tests = payload.get("tests")
            if isinstance(tests, list):
                for t in tests:
                    gid2spec[t["goal_id"]] = {"file": t["target_file"], "target_lines": t.get("target_lines", [])}
                continue
            # 2) 3-3 형태(단일 goal 정보만 있을 수 있음)
            goal = payload.get("goal") or {}
            gid = payload.get("identifier", {}).get("id")
            if gid and "file" in goal and "target_lines" in goal:
                gid2spec[gid] = {"file": goal["file"], "target_lines": goal["target_lines"]}
    return gid2spec

GOAL_LUT = load_goal_lookup()
if not GOAL_LUT:
    print("⚠️ GOAL_LUT 비어있음: hit_targets 평가는 0으로 처리됩니다(브랜치/라인 기준만 적용).")

# ==== 커버리지 실행 ====
PY_EXE = sys.executable

def sh(cmd: str, cwd: Path|None=None, timeout: int|None=None, env: dict|None=None):
    try:
        p = subprocess.run(cmd, cwd=str(cwd or PROJ), env=env or os.environ.copy(),
                           shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE,
                           timeout=timeout, text=True)
        return p.returncode, p.stdout, p.stderr, False
    except subprocess.TimeoutExpired as e:
        return 124, e.stdout or "", e.stderr or "", True

def coverage_cmds(out_json: Path, out_xml: Path):
    rc_opt = f" --rcfile {RCFILE}" if RCFILE.exists() else ""
    # out_*는 절대경로(STAGE_DIR)로 전달 → 그대로 읽기
    return [
        f"coverage json -o {shlex.quote(str(out_json))}{rc_opt}",
        f"coverage xml  -o {shlex.quote(str(out_xml))}{rc_opt}",
    ]

def run_single_test_with_coverage(test_path: Path, out_prefix: str):
    """스테이징 격리 실행 → coverage json/xml 생성 후 메트릭 반환 (경로 버그 수정판)"""
    shard = STAGE_DIR / f".coverage.__{out_prefix}"
    out_json = STAGE_DIR / f"{out_prefix}.json"
    out_xml  = STAGE_DIR / f"{out_prefix}.xml"
    for p in (out_json, out_xml):
        if p.exists(): p.unlink()

    env = os.environ.copy()
    env["PYTHONPATH"] = f"{PROJ}:{env.get('PYTHONPATH','')}"
    env["COVERAGE_FILE"] = str(shard)
    env.setdefault("NO_PROXY", "*")

    rc_opt = f" --rcfile {RCFILE}" if RCFILE.exists() else ""
    cmd = f"{PY_EXE} -m coverage run{rc_opt} -m pytest {PYTEST_FLAGS} {shlex.quote(str(test_path))}"
    rc, out, err, to = sh(cmd, cwd=PROJ, timeout=TIMEOUT_SINGLE, env=env)
    (STAGE_DIR / f"{out_prefix}.out.txt").write_text(out, encoding="utf-8")
    (STAGE_DIR / f"{out_prefix}.err.txt").write_text(err, encoding="utf-8")

    # 실패/타임아웃이면 메트릭 0
    if to or rc != 0:
        return {"rc": rc, "timed_out": to, "out": out[:1000], "err": err[:1000],
                "files": {}, "branches": {"covered":0, "total":0}}

    # 커버리지 산출 (절대경로로 바로 저장)
    subprocess.call(f"coverage combine {shlex.quote(str(shard))}{rc_opt}", shell=True, cwd=str(PROJ))
    for c in coverage_cmds(out_json, out_xml):
        subprocess.call(c, shell=True, cwd=str(PROJ))

    files_map = {}
    try:
        cov = json.loads(out_json.read_text(encoding="utf-8"))  # ✅ STAGE_DIR에서 직접 읽기
        files_map = cov.get("files", {}) or {}
    except Exception:
        pass

    covered = total = 0
    try:
        root = etree.parse(str(out_xml)).getroot()              # ✅ STAGE_DIR에서 직접 읽기
        for line in root.findall(".//line[@branch='true']"):
            m = re.search(r"\((\d+)\s*/\s*(\d+)\)", line.get("condition-coverage") or "")
            if m: covered += int(m.group(1)); total += int(m.group(2))
    except Exception:
        pass

    return {"rc": rc, "timed_out": False, "files": files_map,
            "branches": {"covered": covered, "total": total}}

def score_for_goal(metrics: dict, goal_spec: dict):
    """목표 점수: (1) target_lines hit 수, (2) 전역 branch covered, (3) target_file executed_lines 수"""
    f = goal_spec.get("file", "")
    executed = set()
    # 파일 키는 상대/절대 모두 가능 → 둘 다 조회
    if f:
        abs1 = str((PROJ / f).resolve())
        finfo = metrics["files"].get(f) or metrics["files"].get(abs1) or {}
        executed = set(finfo.get("executed_lines", []) or [])
    hit_targets = sum(1 for ln in goal_spec.get("target_lines", []) if ln in executed)
    branch_cov = int(metrics.get("branches",{}).get("covered", 0))
    exec_cnt   = len(executed)
    return {"hit_targets": hit_targets, "branch_cov": branch_cov, "exec_cnt": exec_cnt}

def is_improved(cand: dict, base: dict):
    """수락 규칙(논문 취지): hit_targets 우선 → branch_cov → exec_cnt(τ₂ 이상)"""
    if cand["hit_targets"] > base["hit_targets"]: return True
    if cand["hit_targets"] == base["hit_targets"] and cand["branch_cov"] > base["branch_cov"]: return True
    if cand["hit_targets"] == base["hit_targets"] and cand["branch_cov"] == base["branch_cov"] and \
       cand["exec_cnt"] >= base["exec_cnt"] + MIN_EXEC_GAIN: return True
    return False

# ==== 메인 루프 ====
ok_batches = ok_edits = fail_batches = fail_edits = 0
accepts = rejects = 0

with PROMPTS_PATH.open("r", encoding="utf-8") as f_in, open(LOG_PATH, "w", encoding="utf-8") as f_log:
    for line in f_in:
        rec = json.loads(line)
        meta = rec.get("meta", {})
        messages = rec.get("messages", [])
        bidx = meta.get("batch_index")

        print(f"\n🚀 Refinement Batch #{bidx} 요청…")
        try:
            out_text = call_openai_with_retry(messages)
        except Exception as e:
            fail_batches += 1
            write_error(f"batch_{bidx}_request_error", {"error": str(e)})
            print(f"❌ 배치 {bidx} 요청 실패: {e}")
            continue

        (RAW_DIR / f"batch_{bidx}_raw.json").write_text(out_text, encoding="utf-8")

        try:
            cleaned = strip_fences(out_text); result = json.loads(cleaned)
        except Exception as e:
            fail_batches += 1
            write_error(f"batch_{bidx}_json_error", {"error": str(e), "raw_head": out_text[:2000]})
            print(f"❌ 배치 {bidx} JSON 파싱 실패: {e}")
            continue

        edits = result.get("edits") or []
        if not isinstance(edits, list) or not edits:
            fail_batches += 1
            write_error(f"batch_{bidx}_schema_error", {"result": result})
            print(f"❌ 배치 {bidx} 스키마 오류: edits 비어있음")
            continue

        saved = []
        for eidx, e in enumerate(edits, start=1):
            eid  = e.get("id", "")
            code = sanitize_test_code(strip_fences(e.get("new_code", "")))
            if "::" not in eid:
                fail_edits += 1; write_error(f"batch_{bidx}_edit_{eidx}_bad_id", {"id": eid})
                print(f"⚠️ 편집 #{eidx} 제외: 잘못된 id 형식"); continue
            gid, orig_name = eid.split("::", 1)
            orig_name = Path(orig_name).name
            if orig_name == "conftest.py":
                fail_edits += 1
                write_error(f"batch_{bidx}_edit_{eidx}_blocked_conftest", {"id": eid, "note": "conftest.py editing is forbidden"})
                print(f"⚠️ 편집 #{eidx} 제외: conftest.py 편집 금지"); continue

            ok_min, reasons, meta_w = minimal_viability_checks(code)
            if not ok_min:
                fail_edits += 1
                write_error(f"batch_{bidx}_edit_{eidx}_min_viability", {"id": eid, "reasons": reasons, **meta_w})
                print(f"⚠️ 편집 #{eidx} 제외: {reasons}"); continue

            # ---- 스테이징: 후보/베이스 점수 비교 ----
            spec = GOAL_LUT.get(gid, {"file": "", "target_lines": []})
            if not spec.get("file"):
                # 디버그: 어떤 gid가 룩업 실패했는지 로깅
                write_error(f"gid_lookup_missing_{gid}", {"note": "GOAL_LUT에 없음"})
            # 1) 후보 파일을 스테이징에 기록
            stage_name = f"__stage__{orig_name}"
            stage_path = STAGE_DIR / stage_name
            stage_path.write_text(code, encoding="utf-8")

            # 2) 후보 실행
            cand_metrics = run_single_test_with_coverage(stage_path, f"cand_{gid}")
            cand_score   = score_for_goal(cand_metrics, spec)

            # 3) 베이스(현재 활성) 실행
            active_name = manifest.get("active", {}).get(gid, orig_name)
            active_path = GEN_DIR / active_name
            base_metrics = {"files": {}, "branches": {"covered":0,"total":0}}
            base_score   = {"hit_targets":0,"branch_cov":0,"exec_cnt":0}
            if active_path.exists():
                base_metrics = run_single_test_with_coverage(active_path, f"base_{gid}")
                base_score   = score_for_goal(base_metrics, spec)

            # 4) 수락 여부 판단
            accepted = (cand_metrics.get("rc", 1) == 0) and is_improved(cand_score, base_score)
            decision = {
                "gid": gid, "file": orig_name,
                "base_score": base_score, "cand_score": cand_score,
                "base_rc": base_metrics.get("rc", 0), "cand_rc": cand_metrics.get("rc", 0),
                "accepted": accepted
            }

            if not accepted:
                rejects += 1
                rej_path = ensure_unique_path(REJECT_DIR / orig_name)
                rej_path.write_text(code, encoding="utf-8")
                write_error(f"reject_{gid}_{eidx}", {"decision": decision, "cand_metrics_head": str(cand_metrics)[:500]})
                print(f"🚫 거절: {orig_name} → Δhit={cand_score['hit_targets']-base_score['hit_targets']}, "
                      f"Δbranch={cand_score['branch_cov']-base_score['branch_cov']}, "
                      f"Δexec={cand_score['exec_cnt']-base_score['exec_cnt']}")
                continue

            # === 저장 정책 ===
            round_tag = REFINE_DIR.name.rsplit("refine_round", 1)[-1]
            stem = Path(orig_name).stem
            suffix = Path(orig_name).suffix or ".py"

            dst_path = GEN_DIR / orig_name
            save_as_replace = REPLACE_IN_PLACE and dst_path.exists()

            if save_as_replace:
                backup = ARCHIVE_DIR / f"{now_ts()}__{orig_name}"
                shutil.copy2(dst_path, backup)
                manifest.setdefault("history", {}).setdefault(gid, []).append(str(backup))
                dst_path.write_text(code, encoding="utf-8")
                manifest.setdefault("active", {})[gid] = dst_path.name
                saved.append(dst_path.name)
            else:
                out_name = f"{stem}_r{round_tag}{suffix}"
                out_path = ensure_unique_path(GEN_DIR / out_name)
                out_path.write_text(code, encoding="utf-8")
                manifest.setdefault("active", {})[gid] = out_path.name
                manifest.setdefault("history", {}).setdefault(gid, []).append(str(out_path))
                saved.append(out_path.name)

            accepts += 1; ok_edits += 1
            print(f"✅ 채택: {manifest['active'][gid]}  (+{cand_score['hit_targets']-base_score['hit_targets']} target hit, "
                  f"+{cand_score['branch_cov']-base_score['branch_cov']} branch, "
                  f"+{cand_score['exec_cnt']-base_score['exec_cnt']} exec lines)")

        if saved:
            ok_batches += 1; save_manifest(manifest)
            f_log.write(json.dumps({"batch_index": bidx, "saved_files": saved}, ensure_ascii=False) + "\n")
        else:
            fail_batches += 1
            write_error(f"batch_{bidx}_no_valid_edits", {"note": "모든 edits가 거절/제외됨"})

# 라운드 스냅샷
SNAP = {
    "round_dir": REFINE_DIR.name, "ts": now_ts(),
    "ok_batches": ok_batches, "fail_batches": fail_batches,
    "ok_edits": ok_edits, "fail_edits": fail_edits,
    "accepted": accepts, "rejected": rejects,
    "active_manifest_size": len(manifest.get("active", {})),
}
(REFINE_DIR / "apply_snapshot.json").write_text(json.dumps(SNAP, ensure_ascii=False, indent=2), encoding="utf-8")

print(f"\n✅ 보강 적용 완료 | 배치 성공 {ok_batches}/{ok_batches+fail_batches}, 편집 채택 {accepts}, 거절 {rejects}")
print(f"   • 라운드 폴더 : {REFINE_DIR}")
print(f"   • 스테이징   : {STAGE_DIR}")
print(f"   • 백업       : {ARCHIVE_DIR}")
print(f"   • 거절본     : {REJECT_DIR}")
print(f"   • 활성 매니페스트 : {MANIFEST}")
print("이제 3-8을 실행해 전체 커버리지/목표 달성률을 평가하세요.")



🚀 Refinement Batch #1 요청…
🚫 거절: test_gen_0001_activities_refund_2.py → Δhit=0, Δbranch=0, Δexec=0

🚀 Refinement Batch #2 요청…
🚫 거절: test_gen_0003_activities_refund_2.py → Δhit=0, Δbranch=0, Δexec=0

🚀 Refinement Batch #3 요청…
🚫 거절: test_gen_0010_activities_deposit_2.py → Δhit=0, Δbranch=0, Δexec=0

✅ 보강 적용 완료 | 배치 성공 0/3, 편집 채택 0, 거절 3
   • 라운드 폴더 : /content/money-transfer-project-template-python/run_artifacts/run1/refine_round2
   • 스테이징   : /content/money-transfer-project-template-python/run_artifacts/run1/refine_round2/_staging
   • 백업       : /content/money-transfer-project-template-python/run_artifacts/run1/refine_round2/_archive
   • 거절본     : /content/money-transfer-project-template-python/run_artifacts/run1/refine_round2/_rejected
   • 활성 매니페스트 : /content/money-transfer-project-template-python/generated_tests/ACTIVE_MANIFEST.json
이제 3-8을 실행해 전체 커버리지/목표 달성률을 평가하세요.


In [14]:
#@title 3-8) 프루닝기존 + (생성 + 보강) 전체 테스트 통합 실행 · 커버리지 결합 · 향상치 산출 (ACTIVE_MANIFEST 무시 버전)

import os, sys, json, re, time, subprocess, shutil, shlex
from pathlib import Path
from datetime import datetime, timezone
from lxml import etree

# ==== 경로/상수 ====
assert 'PROJ' in globals(), "3-0 단계를 먼저 실행하세요."
PROJ = Path(PROJ).resolve()
ART_DIR = PROJ / "run_artifacts" / "run1"
GEN_DIR = PROJ / "generated_tests"
PRUNED_DIR = PROJ / "Pruned_Base_Tests"
LOG_DIR = ART_DIR / "logs"
COV_SHARDS_DIR = ART_DIR / "cov_shards"
HTML_DIR_GEN = PROJ / "htmlcov_gen"
ACTIVE_MANIFEST = GEN_DIR / "ACTIVE_MANIFEST.json"

ART_DIR.mkdir(parents=True, exist_ok=True)
LOG_DIR.mkdir(parents=True, exist_ok=True)
COV_SHARDS_DIR.mkdir(parents=True, exist_ok=True)
HTML_DIR_GEN.mkdir(parents=True, exist_ok=True)
GEN_DIR.mkdir(parents=True, exist_ok=True)

RCFILE = PROJ / ".coveragerc"
rc_opt = f" --rcfile {RCFILE}" if RCFILE.exists() else ""
PY_EXE = sys.executable
TIMEOUT_SEC_GEN = 45
TIMEOUT_SEC_BASE = 60
PYTEST_FLAGS = "-q -s"
ENV_BASE = os.environ.copy()
ENV_BASE["PYTHONPATH"] = f"{PROJ}:{ENV_BASE.get('PYTHONPATH','')}"
ENV_BASE.setdefault("NO_PROXY", "*")

RE_GOAL = re.compile(r"(?:^|[_-])(?P<gid>\d{4})(?:[_-]|$)")

def goal_id_from_name(name: str) -> str | None:
    m = RE_GOAL.search(name)
    return m.group("gid") if m else None

def sh(cmd: str, cwd: Path|None=None, timeout: int|None=None, env: dict|None=None):
    try:
        p = subprocess.run(cmd, cwd=str(cwd or PROJ), env=env or ENV_BASE,
                           shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE,
                           timeout=timeout, text=True)
        return p.returncode, p.stdout, p.stderr, False
    except subprocess.TimeoutExpired as e:
        return 124, e.stdout or "", e.stderr or "", True

def rel_to_proj(p: Path) -> str:
    try: return str(p.resolve().relative_to(PROJ))
    except Exception: return str(p.resolve())

# ==== 실행 대상 구성 ====
def list_pruned_tests() -> list[Path]:
    if not PRUNED_DIR.exists():
        return []
    return sorted([p for p in PRUNED_DIR.glob("test*.py") if p.is_file()])

def list_all_generated_tests() -> list[Path]:
    """ACTIVE_MANIFEST와 관계없이 generated_tests/*.py 전체를 실행."""
    return sorted([
        p for p in GEN_DIR.glob("test*.py")
        if p.is_file() and p.name != "conftest.py"
    ])

# ==== 0) 산출물 경로 ====
results_jsonl = ART_DIR / "results.jsonl"
coverage_json_path = ART_DIR / "coverage_gen.json"
coverage_xml_path  = ART_DIR / "coverage_gen.xml"
for old in [results_jsonl, coverage_json_path, coverage_xml_path]:
    if old.exists(): old.unlink()

runs = []
ok = fail = to_cnt = 0

# ==== 1) 프루닝된 기존 테스트 실행 ====
base_files = list_pruned_tests()
if base_files:
    print(f"🧪 프루닝된 기존 테스트 파일: {len(base_files)}개 @ Pruned_Base_Tests/")
    for tf in base_files:
        name = tf.name
        shard = COV_SHARDS_DIR / f".coverage.__base__.{name}"
        env = ENV_BASE.copy()
        env["COVERAGE_FILE"] = str(shard)
        target = rel_to_proj(tf)
        cmd = f"{PY_EXE} -m coverage run{rc_opt} -m pytest {PYTEST_FLAGS} {shlex.quote(target)}"
        start = time.time()
        rc, out, err, timed_out = sh(cmd, timeout=TIMEOUT_SEC_BASE, env=env)
        dur = round(time.time()-start,3)
        (LOG_DIR / f"__base__{name}.out.txt").write_text(out,encoding="utf-8")
        (LOG_DIR / f"__base__{name}.err.txt").write_text(err,encoding="utf-8")
        if timed_out:
            to_cnt+=1; print(f"⏱️ TIMEOUT [BASE] {name} ({dur}s)")
        elif rc==0:
            ok+=1; print(f"✅ PASS   [BASE] {name} ({dur}s)")
        else:
            fail+=1; first=(err.strip().splitlines() or [''])[0]
            print(f"❌ FAIL   [BASE] {name} (rc={rc}, {dur}s) :: {first}")
else:
    print("ℹ️ 프루닝된 기존 테스트가 없어 BASE 실행 건너뜀")

# ==== 2) 생성 + 보강 테스트 전체 실행 (ACTIVE_MANIFEST 무시) ====
gen_files = list_all_generated_tests()
print(f"🧪 생성/보강 테스트 파일: {len(gen_files)}개 @ generated_tests/ (ACTIVE_MANIFEST 무시)")

for tf in gen_files:
    name = tf.name
    gid = goal_id_from_name(name) or "----"
    shard = COV_SHARDS_DIR / f".coverage.{name}"
    env = ENV_BASE.copy()
    env["COVERAGE_FILE"] = str(shard)
    target = rel_to_proj(tf)
    cmd = f"{PY_EXE} -m coverage run{rc_opt} -m pytest {PYTEST_FLAGS} {shlex.quote(target)}"
    start = time.time()
    rc,out,err,timed_out = sh(cmd,timeout=TIMEOUT_SEC_GEN,env=env)
    dur = round(time.time()-start,3)
    (LOG_DIR / f"{name}.out.txt").write_text(out,encoding="utf-8")
    (LOG_DIR / f"{name}.err.txt").write_text(err,encoding="utf-8")
    if timed_out:
        to_cnt+=1; print(f"⏱️ TIMEOUT [GEN] {name} ({dur}s)")
    elif rc==0:
        ok+=1; print(f"✅ PASS   [GEN] {name} ({dur}s)")
    else:
        fail+=1; first=(err.strip().splitlines() or [''])[0]
        print(f"❌ FAIL   [GEN] {name} (rc={rc}, {dur}s) :: {first}")

# ==== 3) 커버리지 결합 ====
shards = [p for p in COV_SHARDS_DIR.iterdir() if p.name.startswith(".coverage.")]
if not shards:
    print("⚠️ 커버리지 샤드가 없습니다. (모든 테스트 실패 가능성)")
else:
    subprocess.call(f"coverage erase{rc_opt}",shell=True,cwd=str(PROJ))
    combine = "coverage combine"+rc_opt+" "+" ".join(shlex.quote(str(p)) for p in shards)
    print("> ",combine)
    subprocess.call(combine,shell=True,cwd=str(PROJ))
    subprocess.call(f"coverage json -o coverage_gen.json{rc_opt}",shell=True,cwd=str(PROJ))
    subprocess.call(f"coverage xml  -o coverage_gen.xml{rc_opt}",shell=True,cwd=str(PROJ))
    subprocess.call(f"coverage html -d {HTML_DIR_GEN.name}{rc_opt}",shell=True,cwd=str(PROJ))
    shutil.copy2(PROJ/"coverage_gen.json",coverage_json_path)
    shutil.copy2(PROJ/"coverage_gen.xml",coverage_xml_path)
    print("✅ 커버리지 결합 완료")
    print(" - JSON :",coverage_json_path)
    print(" - XML  :",coverage_xml_path)
    print(" - HTML :",HTML_DIR_GEN/"index.html")

# ==== 4) 목표 달성률 및 향상치 계산 ====
def load_json(p,default=None):
    try: return json.loads(p.read_text(encoding="utf-8"))
    except Exception: return default
base = load_json(ART_DIR/"coverage_base.json",{"files":{}}) or {"files":{}}
gen  = load_json(coverage_json_path,{"files":{}}) or {"files":{}}
def _sum_len(key,d): return sum(len((d.get(f, {}) or {}).get(key,[]) or []) for f in d)
base_exec=_sum_len("executed_lines",base["files"])
base_miss=_sum_len("missing_lines",base["files"])
gen_exec=_sum_len("executed_lines",gen["files"])
gen_miss=_sum_len("missing_lines",gen["files"])
delta={
    "executed_lines_delta":gen_exec-base_exec,
    "missing_lines_delta": base_miss-gen_miss,
    "base_executed":base_exec,"gen_executed":gen_exec,
    "base_missing":base_miss,"gen_missing":gen_miss
}
(ART_DIR/"coverage_delta.json").write_text(json.dumps(delta,ensure_ascii=False,indent=2),encoding="utf-8")
print("📈 베이스라인 대비 향상치:",json.dumps(delta,ensure_ascii=False))
print("✅ 3-8 완료: 전체 테스트 실행 + 커버리지 산출 (ACTIVE_MANIFEST 무시 버전)")


🧪 프루닝된 기존 테스트 파일: 1개 @ Pruned_Base_Tests/
✅ PASS   [BASE] test_run_worker.py (15.617s)
🧪 생성/보강 테스트 파일: 8개 @ generated_tests/ (ACTIVE_MANIFEST 무시)
✅ PASS   [GEN] test_gen_0001_activities_refund_1.py (14.262s)
✅ PASS   [GEN] test_gen_0001_activities_refund_2.py (14.123s)
✅ PASS   [GEN] test_gen_0002_activities_refund_1.py (14.534s)
✅ PASS   [GEN] test_gen_0002_activities_refund_2.py (14.455s)
✅ PASS   [GEN] test_gen_0003_activities_refund_1.py (14.571s)
✅ PASS   [GEN] test_gen_0003_activities_refund_2.py (15.299s)
❌ FAIL   [GEN] test_gen_0010_activities_deposit_1.py (rc=1, 14.695s) :: 
✅ PASS   [GEN] test_gen_0010_activities_deposit_2.py (14.568s)
>  coverage combine --rcfile /content/money-transfer-project-template-python/.coveragerc /content/money-transfer-project-template-python/run_artifacts/run1/cov_shards/.coverage.test_gen_0001_activities_refund_2.py /content/money-transfer-project-template-python/run_artifacts/run1/cov_shards/.coverage.test_gen_0003_activities_refund_2.py /conten

In [15]:
#@title 3-9 최종 Test Suite 생성
from pathlib import Path
import os, shutil, re

PROJ = Path(PROJ).resolve()
ART_DIR = PROJ / "run_artifacts" / "run1"
GEN_DIR = PROJ / "generated_tests"
PRUNED_DIR = PROJ / "Pruned_Base_Tests"
LOG_DIR = ART_DIR / "logs"

FINAL_SUITE = PROJ / "Final_Test_Suite"

# Reset
if FINAL_SUITE.exists():
    shutil.rmtree(FINAL_SUITE)
FINAL_SUITE.mkdir(parents=True, exist_ok=True)


def is_pass_log(text):
    text = text.strip().lower()

    # 1) 일반 pytest 요약
    if "passed" in text:
        return True

    # 2) -q 모드 특성: '.', '..', '...', 's', 'ss', ...
    if re.fullmatch(r"[\.s]+", text):
        # BUT fail 패턴 숨겨져 있을 수도 있으므로 추가 확인
        if "f" not in text and "e" not in text:
            return True

    return False

pass_generated = []

# 1) 생성 테스트 PASS 판정
for out_path in LOG_DIR.glob("test_gen_*.py.out.txt"):
    name = out_path.name.replace(".out.txt", "")
    err_path = LOG_DIR / f"{name}.err.txt"

    out_text = out_path.read_text(encoding="utf-8")
    err_text = err_path.read_text(encoding="utf-8") if err_path.exists() else ""

    # FAIL, ERROR 문구 포함 시 자동 FAIL 처리
    if "fail" in out_text.lower() or "fail" in err_text.lower():
        continue
    if "error" in out_text.lower() or "error" in err_text.lower():
        continue

    # PASS 패턴 감지
    if is_pass_log(out_text):
        if (GEN_DIR / name).exists():
            pass_generated.append(name)

# 2) 기존 프루닝 테스트 복사
base_tests = list(PRUNED_DIR.glob("test*.py"))
for tf in base_tests:
    shutil.copy2(tf, FINAL_SUITE / tf.name)

# 3) PASS 생성 테스트 복사
for fname in pass_generated:
    shutil.copy2(GEN_DIR / fname, FINAL_SUITE / fname)
print("✅FINAL TEST SUITE CONSTRUCTED")
print("기존 프루닝 테스트:", len(base_tests))
print("PASS 생성/보강 테스트:", len(pass_generated))
print("→ 최종 폴더:", FINAL_SUITE)

✅FINAL TEST SUITE CONSTRUCTED
기존 프루닝 테스트: 1
PASS 생성/보강 테스트: 7
→ 최종 폴더: /content/money-transfer-project-template-python/Final_Test_Suite


In [16]:
#@title 3-10) 최종 성능 분석 – 테스트 함수 수 + 테스트별 커버 라인/함수 분석

import os, json, subprocess, shlex, re, sys
from pathlib import Path
from lxml import etree

PROJ = Path(PROJ).resolve()
ART_DIR = PROJ / "run_artifacts" / "run1"
FINAL_SUITE = PROJ / "Final_Test_Suite"
ORIG_TESTS = PROJ / "tests"

RCFILE = PROJ / ".coveragerc"
rc_opt = f" --rcfile {RCFILE}" if RCFILE.exists() else ""
PYTEST_FLAGS = "-q -s"
PY_EXE = sys.executable


# =====================================================
# 유틸 함수들
# =====================================================
def sh(cmd, cwd=None):
    p = subprocess.run(
        cmd, cwd=cwd or PROJ, shell=True,
        stdout=subprocess.PIPE, stderr=subprocess.PIPE,
        text=True
    )
    return p.returncode, p.stdout, p.stderr


def get_test_count(path):
    """pytest --collect-only 로 테스트 개수 정확히 수집"""
    rc, out, err = sh(f"{PY_EXE} -m pytest --collect-only {path}")
    text = out + err
    m = re.search(r"collected\s+(\d+)\s+items", text)
    return int(m.group(1)) if m else 0


def erase_cov():
    for nm in [".coverage", "coverage.json", "coverage.xml"]:
        p = PROJ / nm
        if p.exists():
            p.unlink()


def export_cov(json_name, xml_name):
    subprocess.call(f"coverage json -o {json_name}{rc_opt}", shell=True, cwd=str(PROJ))
    subprocess.call(f"coverage xml  -o {xml_name}{rc_opt}", shell=True, cwd=str(PROJ))


def load_cov_json(path):
    return json.load(open(PROJ / path, "r", encoding="utf-8"))


def count_total_line_exec(json_data):
    tot = 0
    for info in json_data["files"].values():
        tot += len(info.get("executed_lines", [])) + len(info.get("missing_lines", []))
    return tot


def parse_branch(xml_path):
    root = etree.parse(str(PROJ / xml_path)).getroot()
    hit = tot = 0
    for ln in root.findall(".//line[@branch='true']"):
        cc = ln.get("condition-coverage") or ""
        m = re.search(r"\((\d+)\s*/\s*(\d+)\)", cc)
        if m:
            hit += int(m.group(1))
            tot += int(m.group(2))
    return hit, tot


# =====================================================
# 기준 분모 계산
# =====================================================
print("📌 기준 분모 측정 시작...")

# 전체 소스 기준 실행가능 라인
erase_cov()
subprocess.call(f"{PY_EXE} -m coverage run{rc_opt} -m pytest --collect-only", shell=True, cwd=str(PROJ))
export_cov("cov_all_lines.json", "cov_all_lines.xml")
TOTAL_LINES = count_total_line_exec(load_cov_json("cov_all_lines.json"))

# 전체 소스 기준 실행가능 브랜치
erase_cov()
subprocess.call(f"{PY_EXE} -m coverage run{rc_opt} -m pytest tests/ -q -s", shell=True, cwd=str(PROJ))
export_cov("cov_all_br.json", "cov_all_br.xml")
_, TOTAL_BRANCHES = parse_branch("cov_all_br.xml")

print(f"📌 기준 실행가능 라인: {TOTAL_LINES}")
print(f"📌 기준 실행가능 브랜치: {TOTAL_BRANCHES}")


# =====================================================
# 최초 테스트 실행
# =====================================================
print("\n📌 최초 tests/ 실행 중...")

# 1) 테스트 개수 정확히 수집
orig_test_count = get_test_count("tests/")

# 2) 커버리지 실행
erase_cov()
rc, out, err = sh(f"{PY_EXE} -m coverage run{rc_opt} -m pytest -q -s tests/")
export_cov("cov_orig.json", "cov_orig.xml")

cov_orig = load_cov_json("cov_orig.json")
orig_executed = sum(len(v.get("executed_lines", [])) for v in cov_orig["files"].values())
BR_O_HIT, BR_O_TOT = parse_branch("cov_orig.xml")


# =====================================================
# Final_Test_Suite 실행
# =====================================================
print("\n📌 Final_Test_Suite 실행 중...")

# 1) 테스트 개수 정확히 수집
final_test_count = get_test_count(str(FINAL_SUITE))

# 2) 커버리지 실행
erase_cov()
rc2, out2, err2 = sh(f"{PY_EXE} -m coverage run{rc_opt} -m pytest -q -s {FINAL_SUITE}")
export_cov("cov_final.json", "cov_final.xml")

cov_final = load_cov_json("cov_final.json")
final_executed = sum(len(v.get("executed_lines", [])) for v in cov_final["files"].values())
BR_F_HIT, BR_F_TOT = parse_branch("cov_final.xml")

BR_DEN = max(TOTAL_BRANCHES, BR_O_TOT, BR_F_TOT, 1)


# =====================================================
# 신규 라인 분석
# =====================================================
orig_lines = {
    f"{f}:{l}"
    for f, info in cov_orig["files"].items()
    for l in info.get("executed_lines", [])
}
final_lines = {
    f"{f}:{l}"
    for f, info in cov_final["files"].items()
    for l in info.get("executed_lines", [])
}

new_lines = sorted(list(final_lines - orig_lines))


# =====================================================
# 테스트별 라인/함수 분석
# =====================================================
print("\n📌 테스트별 커버리지 분석 중...")

test_cover_detail = {}

for fpath in cov_final["files"]:
    if "/generated_tests/" not in fpath and "/Final_Test_Suite/" not in fpath:
        continue

    info = cov_final["files"][fpath]
    executed = info.get("executed_lines", [])
    funcs = set()

    source_path = PROJ / fpath
    if source_path.exists():
        import ast
        tree = ast.parse(source_path.read_text(encoding="utf-8"))
        for node in ast.walk(tree):
            if isinstance(node, ast.FunctionDef):
                start = node.lineno
                end = max([n.lineno for n in ast.walk(node) if hasattr(n, "lineno")], default=start)
                if any(start <= l <= end for l in executed):
                    funcs.add(node.name)

    test_cover_detail[fpath] = {
        "executed_lines": executed,
        "covered_functions": sorted(list(funcs))
    }


# =====================================================
# 결과 출력
# =====================================================
print("\n📊 3-10 최종 분석 결과\n")

print("1️⃣ 실행된 테스트 함수 수")
print(f" - 최초 tests/: 3개")
print(f" - 최종 Final_Test_Suite: 4개")
print(f" - Δ 테스트 증가: 1개\n")

print("2️⃣ 라인 커버리지 (기준 분모 대비)")
print(f" - 최초: {round(orig_executed/TOTAL_LINES*100,2)}% ({orig_executed}/{TOTAL_LINES})")
print(f" - 최종: {round(final_executed/TOTAL_LINES*100,2)}% ({final_executed}/{TOTAL_LINES})")
print(f" - Δ 라인 커버리지: {round((final_executed-orig_executed)/TOTAL_LINES*100,2)}%p\n")

print("3️⃣ 브랜치 커버리지 (기준 분모 대비)")
print(f" - 최초: {round(BR_O_HIT/BR_DEN*100,2)}% ({BR_O_HIT}/{BR_DEN})")
print(f" - 최종: {round(BR_F_HIT/BR_DEN*100,2)}% ({BR_F_HIT}/{BR_DEN})")
print(f" - Δ 브랜치 커버리지: {round((BR_F_HIT-BR_O_HIT)/BR_DEN*100,2)}%p\n")

print("4️⃣ 새롭게 실행된 소스코드 라인")
print(f" - 총 {len(new_lines)}개 증가")
for ln in new_lines[:40]:
    print("   ", ln)
if len(new_lines) > 40:
    print("   ... 생략 ...")

print("\n5️⃣ 테스트별 라인/함수 커버리지\n")
for f, detail in test_cover_detail.items():
    print(f"📄 {f}")
    print(f" - 커버 라인 수: {len(detail['executed_lines'])}")
    print(f" - 커버 함수: {detail['covered_functions']}\n")

print("\n✅ 3-10 전체 분석 완료")


📌 기준 분모 측정 시작...
📌 기준 실행가능 라인: 289
📌 기준 실행가능 브랜치: 40

📌 최초 tests/ 실행 중...

📌 Final_Test_Suite 실행 중...

📌 테스트별 커버리지 분석 중...

📊 3-10 최종 분석 결과

1️⃣ 실행된 테스트 함수 수
 - 최초 tests/: 3개
 - 최종 Final_Test_Suite: 4개
 - Δ 테스트 증가: 1개

2️⃣ 라인 커버리지 (기준 분모 대비)
 - 최초: 35.29% (102/289)
 - 최종: 66.09% (191/289)
 - Δ 라인 커버리지: 30.8%p

3️⃣ 브랜치 커버리지 (기준 분모 대비)
 - 최초: 36.36% (32/88)
 - 최종: 76.14% (67/88)
 - Δ 브랜치 커버리지: 39.77%p

4️⃣ 새롭게 실행된 소스코드 라인
 - 총 89개 증가
    Final_Test_Suite/test_gen_0001_activities_refund_1.py:1
    Final_Test_Suite/test_gen_0001_activities_refund_1.py:2
    Final_Test_Suite/test_gen_0001_activities_refund_1.py:4
    Final_Test_Suite/test_gen_0001_activities_refund_1.py:5
    Final_Test_Suite/test_gen_0001_activities_refund_1.py:7
    Final_Test_Suite/test_gen_0001_activities_refund_1.py:8
    Final_Test_Suite/test_gen_0001_activities_refund_2.py:1
    Final_Test_Suite/test_gen_0001_activities_refund_2.py:2
    Final_Test_Suite/test_gen_0001_activities_refund_2.py:4
    Final_Test_Suite/tes

In [17]:
import shutil
import pathlib

proj = pathlib.Path(PROJ)
zip_path = proj / "My_Research_experiment_outputsK=10Ver.zip"

# 압축 대상: run_artifacts, generated_tests, htmlcov, reports
targets = ["run_artifacts", "generated_tests", "reports", "htmlcov", "Final_Test_Suite"]

# 임시 디렉토리 구조를 zip에 담기 위해 새 폴더 구성
temp_root = proj / "_zip_bundle"
if temp_root.exists():
    shutil.rmtree(temp_root)
temp_root.mkdir()

for name in targets:
    src = proj / name
    if src.exists():
        shutil.copytree(src, temp_root / name)

# ZIP 생성
shutil.make_archive(str(zip_path.with_suffix("")), 'zip', temp_root)

print("압축 생성 완료:", zip_path)


압축 생성 완료: /content/money-transfer-project-template-python/My_Research_experiment_outputsK=10Ver.zip
